15 sku

In [1]:
# =========================================================
# SARIMAX BASELINE (DATASET 15) - EVENT / HOLIDAY / RAINFALL
# Grid search:
#   - order, seasonal_order
#   - kombinasi exog:
#       (1) none
#       (2) event_flag
#       (3) event_flag + holiday_count
#       (4) event_flag + holiday_count + rainfall_lag1
#   - versi log1p vs level
# d_suggest diambil dari model_profiles_selected15.csv
# =========================================================
import itertools
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import acorr_ljungbox

warnings.filterwarnings("ignore")

# ---------------- PATH ----------------
PROJECT_ROOT = Path(r"D:\Documents\Skripsi\demand-forecasting")

DATA_PATH    = PROJECT_ROOT / "data" / "dataset_15" / "sarimax_selected15_series.csv"
PROFILE_PATH = PROJECT_ROOT / "data" / "dataset_15" / "model_profiles_selected15.csv"

OUT_DIR   = PROJECT_ROOT / "outputs" / "sarimax_selected15"
CHART_DIR = OUT_DIR / "charts"
OUT_DIR.mkdir(parents=True, exist_ok=True)
CHART_DIR.mkdir(parents=True, exist_ok=True)

# ---------------- LOAD ----------------
df = pd.read_csv(DATA_PATH, parse_dates=["periode"]).sort_values(
    ["cabang", "sku", "periode"]
)

profiles = pd.read_csv(PROFILE_PATH)

# map d_suggest per (cabang, sku) dari model_profiles
if "sarimax_d_sug" in profiles.columns:
    d_suggest_map = dict(
        zip(
            zip(profiles["cabang"], profiles["sku"]),
            profiles["sarimax_d_sug"].astype(int),
        )
    )
else:
    print("[WARN] Kolom sarimax_d_sug tidak ada di model_profiles. Pakai d=1 semua.")
    d_suggest_map = {}

# ---------------- QUICK SANITY ----------------
expected_cols = [
    "cabang",
    "sku",
    "periode",
    "qty",
    "event_flag",
    "is_train",
    "is_test",
]
missing = [c for c in expected_cols if c not in df.columns]
if missing:
    raise ValueError(f"Kolom wajib hilang: {missing}")

dups = df.duplicated(subset=["cabang", "sku", "periode"]).sum()
if dups:
    raise ValueError(f"Duplikat (cabang, sku, periode): {dups} baris")

test_total = int((df["is_test"] == 1).sum())
print(f"[INFO] Total baris TEST: {test_total}")

for c in ["qty", "event_flag", "holiday_count", "rainfall_lag1"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# ---------------- HELPERS ----------------
def mape(a, f):
    a, f = np.asarray(a, float), np.asarray(f, float)
    mask = a != 0
    return float(np.mean(np.abs((a[mask] - f[mask]) / a[mask])) * 100) if mask.any() else np.nan

def smape(a, f):
    a, f = np.asarray(a, float), np.asarray(f, float)
    denom = (np.abs(a) + np.abs(f)) / 2.0
    mask = denom != 0
    return float(np.mean(np.abs(a[mask] - f[mask]) / denom[mask]) * 100) if mask.any() else np.nan

def param_grid_small(d_fix=None):
    """
    Grid kecil untuk (p,d,q) dan (P,D,Q,12).
    Kalau d_fix tidak None, pakai hanya d_fix.
    """
    yielded = set()
    d_values = [0, 1] if d_fix is None else [int(d_fix)]

    for p, d, q in itertools.product([0, 1, 2], d_values, [0, 1, 2]):
        for P, D, Q in itertools.product([0, 1], [0, 1], [0, 1]):
            if (p + q + P + Q) <= 6:
                od, sod = (p, d, q), (P, D, Q, 12)
                if (od, sod) in yielded:
                    continue
                yielded.add((od, sod))
                yield od, sod

def fit_eval(y_tr, X_tr, y_te, X_te, order, sorder, use_log=False):
    # transform target (opsional)
    ytr = np.log1p(np.clip(y_tr, a_min=0, a_max=None)) if use_log else y_tr

    model = SARIMAX(
        ytr,
        exog=X_tr,
        order=order,
        seasonal_order=sorder,
        enforce_stationarity=False,
        enforce_invertibility=False,
    )
    res = model.fit(disp=False)

    # fitted (train)
    fitted_tr = res.fittedvalues
    # jaga panjang kalau trimming karena NaN awal
    n_tr = min(len(y_tr), len(fitted_tr))
    y_tr_cut = y_tr[-n_tr:]
    fitted_tr = np.asarray(fitted_tr[-n_tr:], float)

    rmse_train = float(np.sqrt(np.mean((y_tr_cut - fitted_tr) ** 2)))
    mae_train  = float(np.mean(np.abs(y_tr_cut - fitted_tr)))
    mape_train = mape(y_tr_cut, fitted_tr)
    smape_train = smape(y_tr_cut, fitted_tr)

    # forecast untuk test (kalau ada)
    if y_te is not None and len(y_te) > 0:
        fh = len(y_te)
        pred_t = res.get_forecast(steps=fh, exog=X_te).predicted_mean
        yhat = np.expm1(pred_t) if use_log else np.asarray(pred_t, float)

        rmse_test = float(np.sqrt(np.mean((y_te - yhat) ** 2)))
        mae_test  = float(np.mean(np.abs(y_te - yhat)))
        mape_test = mape(y_te, yhat)
        smape_test = smape(y_te, yhat)
    else:
        yhat = np.array([], float)
        rmse_test = np.nan
        mae_test  = np.nan
        mape_test = np.nan
        smape_test = np.nan

    resid = res.resid
    try:
        lb_p = float(
            acorr_ljungbox(resid, lags=[12], return_df=True)["lb_pvalue"].iloc[-1]
        )
    except Exception:
        lb_p = np.nan

    return dict(
        order=order,
        seasonal_order=sorder,
        AIC_train=float(res.aic),

        RMSE_train=rmse_train,
        MAE_train=mae_train,
        MAPE_train=mape_train,
        sMAPE_train=smape_train,

        RMSE_test=rmse_test,
        MAE_test=mae_test,
        MAPE_test=mape_test,
        sMAPE_test=smape_test,

        LB_p12=lb_p,
        yhat=yhat,
    )

def grid_fit_series(g, exog_cols=None, use_log=False, d_fix=None):
    g = g.sort_values("periode").copy()
    y = g["qty"].astype(float).values
    X = g[exog_cols].astype(float).values if exog_cols else None

    m_tr = g["is_train"].astype(bool).values
    m_te = g["is_test"].astype(bool).values

    y_tr = y[m_tr]
    y_te = y[m_te]
    X_tr = X[m_tr] if X is not None else None
    X_te = X[m_te] if X is not None else None

    # bersihkan NaN di train
    if X_tr is not None:
        ok = ~np.isnan(y_tr)
        y_tr, X_tr = y_tr[ok], X_tr[ok]
        ok = ~np.isnan(X_tr).any(axis=1)
        y_tr, X_tr = y_tr[ok], X_tr[ok]
    else:
        y_tr = y_tr[~np.isnan(y_tr)]

    if len(y_tr) < 24:
        return None, pd.DataFrame()

    # untuk test, kalau tidak ada, biarkan kosong saja
    if X_te is not None and len(X_te) > 0:
        # buang baris NaN di exog test (harusnya sudah rapi)
        ok_te = ~np.isnan(X_te).any(axis=1)
        y_te = y_te[ok_te]
        X_te = X_te[ok_te]

    tried, best = [], None
    for od, sod in param_grid_small(d_fix=d_fix):
        try:
            rec = fit_eval(y_tr, X_tr, y_te, X_te, od, sod, use_log=use_log)
            tried.append(
                {
                    k: rec[k]
                    for k in [
                        "order",
                        "seasonal_order",
                        "AIC_train",
                        "RMSE_train",
                        "MAE_train",
                        "MAPE_train",
                        "sMAPE_train",
                        "RMSE_test",
                        "MAE_test",
                        "MAPE_test",
                        "sMAPE_test",
                        "LB_p12",
                    ]
                }
            )
            # ranking:
            # 1) kalau ada test → pakai RMSE_test + AIC_train
            # 2) kalau tidak ada test → pakai RMSE_train + AIC_train
            if best is None:
                best = rec
            else:
                if not np.isnan(rec["RMSE_test"]) and not np.isnan(best["RMSE_test"]):
                    better = (rec["RMSE_test"] < best["RMSE_test"]) or (
                        rec["RMSE_test"] == best["RMSE_test"]
                        and rec["AIC_train"] < best["AIC_train"]
                    )
                elif np.isnan(rec["RMSE_test"]) and np.isnan(best["RMSE_test"]):
                    better = (rec["RMSE_train"] < best["RMSE_train"]) or (
                        rec["RMSE_train"] == best["RMSE_train"]
                        and rec["AIC_train"] < best["AIC_train"]
                    )
                else:
                    # kalau salah satu tidak punya test, prioritaskan yang punya test
                    better = not np.isnan(rec["RMSE_test"]) and np.isnan(best["RMSE_test"])

                if better:
                    best = rec

        except Exception:
            continue

    tried_df = (
        pd.DataFrame(tried)
        .sort_values(["RMSE_test", "AIC_train"])
        .reset_index(drop=True)
        if tried
        else pd.DataFrame()
    )
    return best, tried_df

def plot_test_series(preds_df, cabang, sku, out_dir: Path):
    d = preds_df[(preds_df["cabang"] == cabang) & (preds_df["sku"] == sku)].copy()
    if d.empty:
        return
    d = d.sort_values("periode")
    d["periode"] = pd.to_datetime(d["periode"])
    rmse_val = float(np.sqrt(np.mean((d["qty"].values - d["pred"].values) ** 2)))

    fig, ax = plt.subplots(figsize=(10.5, 3.6))
    locator = mdates.MonthLocator(interval=1)
    ax.xaxis.set_major_locator(locator)
    ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(locator))
    ax.plot(d["periode"], d["qty"], lw=1.8, marker="o", label="Actual")
    ax.plot(d["periode"], d["pred"], lw=1.8, marker="s", label="Pred")
    ax.set_title(f"{cabang}-{sku} | RMSE_test={rmse_val:.1f}")
    ax.set_xlabel("Periode")
    ax.set_ylabel("Qty")
    ax.grid(True, axis="y", alpha=0.25)
    ax.legend(loc="center left", bbox_to_anchor=(1.02, 0.5), frameon=False)
    fig.tight_layout(rect=[0, 0, 0.82, 1])
    fpath = out_dir / f"test_plot__{cabang}__{sku}.png"
    fig.savefig(fpath, dpi=160)
    plt.close(fig)

# ---------------- LOOP PER SERI ----------------
records, preds = [], []

# DI SINI BEDANYA:
# bukan hanya sku yang punya is_test, tapi semua sku di dataset 15
pairs = (
    df[["cabang", "sku"]]
    .drop_duplicates()
    .itertuples(index=False, name=None)
)

for cab, sku in pairs:
    g = df[(df["cabang"] == cab) & (df["sku"] == sku)].copy()

    d_val = int(d_suggest_map.get((cab, sku), 1))

    exog_candidates = []
    exog_candidates.append(("none", []))
    if "event_flag" in g.columns:
        exog_candidates.append(("event", ["event_flag"]))
    if {"event_flag", "holiday_count"}.issubset(g.columns):
        exog_candidates.append(("event_hol", ["event_flag", "holiday_count"]))
    if {"event_flag", "holiday_count", "rainfall_lag1"}.issubset(g.columns):
        exog_candidates.append(
            (
                "event_hol_rain",
                ["event_flag", "holiday_count", "rainfall_lag1"],
            )
        )

    best_var = None
    best_rec = None
    best_exog_cols = []

    # cari kombinasi exog terbaik (tanpa log dulu)
    for tag, cols in exog_candidates:
        b, _ = grid_fit_series(
            g,
            exog_cols=cols if cols else None,
            use_log=False,
            d_fix=d_val,
        )
        if not b:
            continue
        if best_rec is None:
            best_var = tag
            best_rec = b
            best_exog_cols = cols
        else:
            if not np.isnan(b["RMSE_test"]) and not np.isnan(best_rec["RMSE_test"]):
                better = (b["RMSE_test"] < best_rec["RMSE_test"]) or (
                    b["RMSE_test"] == best_rec["RMSE_test"]
                    and b["AIC_train"] < best_rec["AIC_train"]
                )
            elif np.isnan(b["RMSE_test"]) and np.isnan(best_rec["RMSE_test"]):
                better = (b["RMSE_train"] < best_rec["RMSE_train"]) or (
                    b["RMSE_train"] == best_rec["RMSE_train"]
                    and b["AIC_train"] < best_rec["AIC_train"]
                )
            else:
                better = not np.isnan(b["RMSE_test"]) and np.isnan(best_rec["RMSE_test"])

            if better:
                best_var = tag
                best_rec = b
                best_exog_cols = cols

    if best_rec is None:
        print(f"[SKIP] {cab}-{sku}: gagal fit (train terlalu pendek atau error).")
        continue

    # coba versi log1p dengan exog terbaik
    blog, _ = grid_fit_series(
        g,
        exog_cols=best_exog_cols if best_exog_cols else None,
        use_log=True,
        d_fix=d_val,
    )
    use_log = False
    cur = best_rec
    variant_tag = best_var

    if blog and (
        (not np.isnan(blog["RMSE_test"]) and not np.isnan(cur["RMSE_test"]) and blog["RMSE_test"] < cur["RMSE_test"])
        or (
            np.isnan(blog["RMSE_test"]) and np.isnan(cur["RMSE_test"])
            and blog["RMSE_train"] < cur["RMSE_train"]
        )
    ):
        cur = blog
        use_log = True
        variant_tag = best_var + "+log"

    records.append(
        {
            "cabang": cab,
            "sku": sku,
            "variant": variant_tag,
            "exog_used": ",".join(best_exog_cols) if best_exog_cols else "(none)",
            "use_log": use_log,

            "order": cur["order"],
            "seasonal_order": cur["seasonal_order"],
            "AIC_train": cur["AIC_train"],

            "RMSE_train": cur["RMSE_train"],
            "MAE_train": cur["MAE_train"],
            "MAPE%_train": cur["MAPE_train"],
            "sMAPE%_train": cur["sMAPE_train"],

            "RMSE_test": cur["RMSE_test"],
            "MAE_test": cur["MAE_test"],
            "MAPE%_test": cur["MAPE_test"],
            "sMAPE%_test": cur["sMAPE_test"],

            "LB_p12": cur["LB_p12"],
        }
    )

    # pred hanya buat seri yang punya test
    m_te = g["is_test"] == 1
    if m_te.any() and len(cur["yhat"]) > 0:
        preds.append(
            pd.DataFrame(
                {
                    "periode": g.loc[m_te, "periode"].values,
                    "qty": g.loc[m_te, "qty"].values,
                    "pred": np.asarray(cur["yhat"], float),
                    "cabang": cab,
                    "sku": sku,
                }
            )
        )

# ---------------- OUTPUT & SAVE ----------------
results_df = (
    pd.DataFrame(records)
    .sort_values(["cabang", "sku"])
    .reset_index(drop=True)
)
forecasts_df = (
    pd.concat(preds, ignore_index=True) if len(preds) else pd.DataFrame()
)

print("\n=== RINGKASAN HASIL (DATASET 15) - BEBERAPA BARIS ===")
if not results_df.empty:
    print(
        results_df[
            [
                "cabang",
                "sku",
                "variant",
                "exog_used",
                "order",
                "seasonal_order",
                "RMSE_train",
                "RMSE_test",
                "LB_p12",
            ]
        ].head(20).to_string(index=False)
    )
else:
    print("Tidak ada hasil. Seri terlalu pendek atau split bermasalah.")

METRICS_PATH = OUT_DIR / "sarimax_selected15_metrics.csv"
PREDS_PATH   = OUT_DIR / "sarimax_selected15_predictions.csv"
results_df.to_csv(METRICS_PATH, index=False)
forecasts_df.to_csv(PREDS_PATH, index=False)
print(f"\nSaved metrics  -> {METRICS_PATH}")
print(f"Saved preds    -> {PREDS_PATH}")

# CHARTS (hanya seri dengan test)
if not forecasts_df.empty:
    for cab, sk in (
        forecasts_df[["cabang", "sku"]].drop_duplicates().itertuples(
            index=False, name=None
        )
    ):
        plot_test_series(forecasts_df, cab, sk, CHART_DIR)
    print(f"Saved charts   -> {CHART_DIR}")
else:
    print("Tidak ada prediksi test untuk digambar.")


[INFO] Total baris TEST: 45

=== RINGKASAN HASIL (DATASET 15) - BEBERAPA BARIS ===
cabang          sku       variant                exog_used     order seasonal_order  RMSE_train   RMSE_test       LB_p12
   02A  BNOP400CHAR         event               event_flag (2, 1, 1)  (0, 0, 0, 12)  665.621129         NaN 9.622454e-01
   02A  BNOP400CPOX         event               event_flag (2, 1, 1)  (0, 0, 0, 12)  665.621129         NaN 9.622454e-01
   02A  BUVW001K194     event_hol event_flag,holiday_count (2, 0, 1)  (0, 0, 0, 12)  383.261847         NaN 3.380048e-01
   02A   BUVW001KSB     event_hol event_flag,holiday_count (2, 0, 1)  (0, 0, 0, 12)  547.539115         NaN 8.157573e-01
   02A  BUVW001KSBM          none                   (none) (2, 0, 1)  (0, 0, 0, 12)  348.688982         NaN 8.415164e-01
   02A   BUVW001KSW event_hol+log event_flag,holiday_count (1, 1, 0)  (1, 1, 1, 12) 4286.438241 1551.313761 2.157863e-01
   02A  BUVW100C192          none                   (none) (2, 0, 1)  

In [2]:
# =========================================================
# SARIMAX BASELINE (DATASET 15) - EVENT / HOLIDAY / RAINFALL
# Grid search:
#   - order, seasonal_order
#   - kombinasi exog:
#       (1) none
#       (2) event_flag
#       (3) event_flag + holiday_count
#       (4) event_flag + holiday_count + rainfall_lag1
#   - versi log1p vs level
# d_suggest diambil dari model_profiles_selected15.csv
# =========================================================
import itertools
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import acorr_ljungbox

warnings.filterwarnings("ignore")

# ---------------- PATH ----------------
PROJECT_ROOT = Path(r"D:\Documents\Skripsi\demand-forecasting")

DATA_PATH    = PROJECT_ROOT / "data" / "dataset_15" / "sarimax_selected15_series.csv"
PROFILE_PATH = PROJECT_ROOT / "data" / "dataset_15" / "model_profiles_selected15.csv"

OUT_DIR   = PROJECT_ROOT / "outputs" / "sarimax_selected15"
CHART_DIR = OUT_DIR / "charts"
OUT_DIR.mkdir(parents=True, exist_ok=True)
CHART_DIR.mkdir(parents=True, exist_ok=True)

# ---------------- LOAD ----------------
df = pd.read_csv(DATA_PATH, parse_dates=["periode"]).sort_values(
    ["cabang", "sku", "periode"]
)

profiles = pd.read_csv(PROFILE_PATH)

# map d_suggest per (cabang, sku) dari model_profiles
if "sarimax_d_sug" in profiles.columns:
    d_suggest_map = dict(
        zip(
            zip(profiles["cabang"], profiles["sku"]),
            profiles["sarimax_d_sug"].astype(int),
        )
    )
else:
    print("[WARN] Kolom sarimax_d_sug tidak ada di model_profiles. Pakai d=1 semua.")
    d_suggest_map = {}

# ---------------- QUICK SANITY ----------------
expected_cols = [
    "cabang",
    "sku",
    "periode",
    "qty",
    "event_flag",
    "is_train",
    "is_test",
]
missing = [c for c in expected_cols if c not in df.columns]
if missing:
    raise ValueError(f"Kolom wajib hilang: {missing}")

dups = df.duplicated(subset=["cabang", "sku", "periode"]).sum()
if dups:
    raise ValueError(f"Duplikat (cabang, sku, periode): {dups} baris")

test_total = int((df["is_test"] == 1).sum())
print(f"[INFO] Total baris TEST: {test_total}")

for c in ["qty", "event_flag", "holiday_count", "rainfall_lag1"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# ---------------- HELPERS ----------------
def mape(a, f):
    a, f = np.asarray(a, float), np.asarray(f, float)
    mask = a != 0
    return float(np.mean(np.abs((a[mask] - f[mask]) / a[mask])) * 100) if mask.any() else np.nan

def smape(a, f):
    a, f = np.asarray(a, float), np.asarray(f, float)
    denom = (np.abs(a) + np.abs(f)) / 2.0
    mask = denom != 0
    return float(np.mean(np.abs(a[mask] - f[mask]) / denom[mask]) * 100) if mask.any() else np.nan

def param_grid_small(d_fix=None):
    """
    Grid kecil untuk (p,d,q) dan (P,D,Q,12).
    Kalau d_fix tidak None, pakai hanya d_fix.
    """
    yielded = set()
    d_values = [0, 1] if d_fix is None else [int(d_fix)]

    for p, d, q in itertools.product([0, 1, 2], d_values, [0, 1, 2]):
        for P, D, Q in itertools.product([0, 1], [0, 1], [0, 1]):
            if (p + q + P + Q) <= 6:
                od, sod = (p, d, q), (P, D, Q, 12)
                if (od, sod) in yielded:
                    continue
                yielded.add((od, sod))
                yield od, sod

def fit_eval(y_tr, X_tr, y_te, X_te, order, sorder, use_log=False):
    # transform target (opsional)
    ytr = np.log1p(np.clip(y_tr, a_min=0, a_max=None)) if use_log else y_tr

    model = SARIMAX(
        ytr,
        exog=X_tr,
        order=order,
        seasonal_order=sorder,
        enforce_stationarity=False,
        enforce_invertibility=False,
    )
    res = model.fit(disp=False)

    # fitted (train)
    fitted_tr = res.fittedvalues
    n_tr = min(len(y_tr), len(fitted_tr))
    y_tr_cut = y_tr[-n_tr:]
    fitted_tr = np.asarray(fitted_tr[-n_tr:], float)

    rmse_train = float(np.sqrt(np.mean((y_tr_cut - fitted_tr) ** 2)))
    mae_train  = float(np.mean(np.abs(y_tr_cut - fitted_tr)))
    mape_train = mape(y_tr_cut, fitted_tr)
    sMAPE_train = smape(y_tr_cut, fitted_tr)

    # forecast untuk test (kalau ada)
    if y_te is not None and len(y_te) > 0:
        fh = len(y_te)
        pred_t = res.get_forecast(steps=fh, exog=X_te).predicted_mean
        yhat = np.expm1(pred_t) if use_log else np.asarray(pred_t, float)

        rmse_test = float(np.sqrt(np.mean((y_te - yhat) ** 2)))
        mae_test  = float(np.mean(np.abs(y_te - yhat)))
        mape_test = mape(y_te, yhat)
        sMAPE_test = smape(y_te, yhat)
    else:
        yhat = np.array([], float)
        rmse_test = np.nan
        mae_test  = np.nan
        mape_test = np.nan
        sMAPE_test = np.nan

    resid = res.resid
    try:
        lb_p = float(
            acorr_ljungbox(resid, lags=[12], return_df=True)["lb_pvalue"].iloc[-1]
        )
    except Exception:
        lb_p = np.nan

    return dict(
        order=order,
        seasonal_order=sorder,
        AIC_train=float(res.aic),

        RMSE_train=rmse_train,
        MAE_train=mae_train,
        MAPE_train=mape_train,
        sMAPE_train=sMAPE_train,

        RMSE_test=rmse_test,
        MAE_test=mae_test,
        MAPE_test=mape_test,
        sMAPE_test=sMAPE_test,

        LB_p12=lb_p,
        yhat=yhat,
    )

def _better_by_train(rec, best):
    """
    Bandingkan dua kandidat pakai:
      1) RMSE_train (lebih kecil lebih baik)
      2) AIC_train sebagai tie-break
    """
    if best is None:
        return True

    r1 = rec["RMSE_train"]
    r2 = best["RMSE_train"]

    if np.isnan(r2):
        return True
    if np.isnan(r1):
        return False

    if r1 < r2:
        return True
    if r1 > r2:
        return False

    # tie di RMSE_train, pakai AIC_train
    return rec["AIC_train"] < best["AIC_train"]

def grid_fit_series(g, exog_cols=None, use_log=False, d_fix=None):
    g = g.sort_values("periode").copy()
    y = g["qty"].astype(float).values
    X = g[exog_cols].astype(float).values if exog_cols else None

    m_tr = g["is_train"].astype(bool).values
    m_te = g["is_test"].astype(bool).values

    y_tr = y[m_tr]
    y_te = y[m_te]
    X_tr = X[m_tr] if X is not None else None
    X_te = X[m_te] if X is not None else None

    # bersihkan NaN di train
    if X_tr is not None:
        ok = ~np.isnan(y_tr)
        y_tr, X_tr = y_tr[ok], X_tr[ok]
        ok = ~np.isnan(X_tr).any(axis=1)
        y_tr, X_tr = y_tr[ok], X_tr[ok]
    else:
        y_tr = y_tr[~np.isnan(y_tr)]

    if len(y_tr) < 24:
        return None, pd.DataFrame()

    if X_te is not None and len(X_te) > 0:
        ok_te = ~np.isnan(X_te).any(axis=1)
        y_te = y_te[ok_te]
        X_te = X_te[ok_te]

    tried, best = [], None
    for od, sod in param_grid_small(d_fix=d_fix):
        try:
            rec = fit_eval(y_tr, X_tr, y_te, X_te, od, sod, use_log=use_log)
            tried.append(
                {
                    k: rec[k]
                    for k in [
                        "order",
                        "seasonal_order",
                        "AIC_train",
                        "RMSE_train",
                        "MAE_train",
                        "MAPE_train",
                        "sMAPE_train",
                        "RMSE_test",
                        "MAE_test",
                        "MAPE_test",
                        "sMAPE_test",
                        "LB_p12",
                    ]
                }
            )

            if _better_by_train(rec, best):
                best = rec

        except Exception:
            continue

    tried_df = (
        pd.DataFrame(tried)
        .sort_values(["RMSE_train", "AIC_train"])
        .reset_index(drop=True)
        if tried
        else pd.DataFrame()
    )
    return best, tried_df

def plot_test_series(preds_df, cabang, sku, out_dir: Path):
    d = preds_df[(preds_df["cabang"] == cabang) & (preds_df["sku"] == sku)].copy()
    if d.empty:
        return
    d = d.sort_values("periode")
    d["periode"] = pd.to_datetime(d["periode"])
    rmse_val = float(np.sqrt(np.mean((d["qty"].values - d["pred"].values) ** 2)))

    fig, ax = plt.subplots(figsize=(10.5, 3.6))
    locator = mdates.MonthLocator(interval=1)
    ax.xaxis.set_major_locator(locator)
    ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(locator))
    ax.plot(d["periode"], d["qty"], lw=1.8, marker="o", label="Actual")
    ax.plot(d["periode"], d["pred"], lw=1.8, marker="s", label="Pred")
    ax.set_title(f"{cabang}-{sku} | RMSE_test={rmse_val:.1f}")
    ax.set_xlabel("Periode")
    ax.set_ylabel("Qty")
    ax.grid(True, axis="y", alpha=0.25)
    ax.legend(loc="center left", bbox_to_anchor=(1.02, 0.5), frameon=False)
    fig.tight_layout(rect=[0, 0, 0.82, 1])
    fpath = out_dir / f"test_plot__{cabang}__{sku}.png"
    fig.savefig(fpath, dpi=160)
    plt.close(fig)

# ---------------- LOOP PER SERI ----------------
records, preds = [], []

pairs = (
    df[["cabang", "sku"]]
    .drop_duplicates()
    .itertuples(index=False, name=None)
)

for cab, sku in pairs:
    g = df[(df["cabang"] == cab) & (df["sku"] == sku)].copy()

    d_val = int(d_suggest_map.get((cab, sku), 1))

    exog_candidates = []
    exog_candidates.append(("none", []))
    if "event_flag" in g.columns:
        exog_candidates.append(("event", ["event_flag"]))
    if {"event_flag", "holiday_count"}.issubset(g.columns):
        exog_candidates.append(("event_hol", ["event_flag", "holiday_count"]))
    if {"event_flag", "holiday_count", "rainfall_lag1"}.issubset(g.columns):
        exog_candidates.append(
            (
                "event_hol_rain",
                ["event_flag", "holiday_count", "rainfall_lag1"],
            )
        )

    best_var = None
    best_rec = None
    best_exog_cols = []

    # cari kombinasi exog terbaik (pakai RMSE_train + AIC_train)
    for tag, cols in exog_candidates:
        b, _ = grid_fit_series(
            g,
            exog_cols=cols if cols else None,
            use_log=False,
            d_fix=d_val,
        )
        if not b:
            continue
        if _better_by_train(b, best_rec):
            best_var = tag
            best_rec = b
            best_exog_cols = cols

    if best_rec is None:
        print(f"[SKIP] {cab}-{sku}: gagal fit (train terlalu pendek atau error).")
        continue

    # coba versi log1p dengan exog terbaik
    blog, _ = grid_fit_series(
        g,
        exog_cols=best_exog_cols if best_exog_cols else None,
        use_log=True,
        d_fix=d_val,
    )
    use_log = False
    cur = best_rec
    variant_tag = best_var

    if blog and _better_by_train(blog, cur):
        cur = blog
        use_log = True
        variant_tag = best_var + "+log"

    records.append(
        {
            "cabang": cab,
            "sku": sku,
            "variant": variant_tag,
            "exog_used": ",".join(best_exog_cols) if best_exog_cols else "(none)",
            "use_log": use_log,

            "order": cur["order"],
            "seasonal_order": cur["seasonal_order"],
            "AIC_train": cur["AIC_train"],

            "RMSE_train": cur["RMSE_train"],
            "MAE_train": cur["MAE_train"],
            "MAPE%_train": cur["MAPE_train"],
            "sMAPE%_train": cur["sMAPE_train"],

            "RMSE_test": cur["RMSE_test"],
            "MAE_test": cur["MAE_test"],
            "MAPE%_test": cur["MAPE_test"],
            "sMAPE%_test": cur["sMAPE_test"],

            "LB_p12": cur["LB_p12"],
        }
    )

    # pred hanya buat seri yang punya test
    m_te = g["is_test"] == 1
    if m_te.any() and len(cur["yhat"]) > 0:
        preds.append(
            pd.DataFrame(
                {
                    "periode": g.loc[m_te, "periode"].values,
                    "qty": g.loc[m_te, "qty"].values,
                    "pred": np.asarray(cur["yhat"], float),
                    "cabang": cab,
                    "sku": sku,
                }
            )
        )

# ---------------- OUTPUT & SAVE ----------------
results_df = (
    pd.DataFrame(records)
    .sort_values(["cabang", "sku"])
    .reset_index(drop=True)
)
forecasts_df = (
    pd.concat(preds, ignore_index=True) if len(preds) else pd.DataFrame()
)

print("\n=== RINGKASAN HASIL (DATASET 15) - BEBERAPA BARIS ===")
if not results_df.empty:
    print(
        results_df[
            [
                "cabang",
                "sku",
                "variant",
                "exog_used",
                "order",
                "seasonal_order",
                "RMSE_train",
                "RMSE_test",
                "LB_p12",
            ]
        ].head(20).to_string(index=False)
    )
else:
    print("Tidak ada hasil. Seri terlalu pendek atau split bermasalah.")

METRICS_PATH = OUT_DIR / "sarimax_selected15_metrics.csv"
PREDS_PATH   = OUT_DIR / "sarimax_selected15_predictions.csv"
results_df.to_csv(METRICS_PATH, index=False)
forecasts_df.to_csv(PREDS_PATH, index=False)
print(f"\nSaved metrics  -> {METRICS_PATH}")
print(f"Saved preds    -> {PREDS_PATH}")

# CHARTS (hanya seri dengan test)
if not forecasts_df.empty:
    for cab, sk in (
        forecasts_df[["cabang", "sku"]].drop_duplicates().itertuples(
            index=False, name=None
        )
    ):
        plot_test_series(forecasts_df, cab, sk, CHART_DIR)
    print(f"Saved charts   -> {CHART_DIR}")
else:
    print("Tidak ada prediksi test untuk digambar.")


[INFO] Total baris TEST: 45

=== RINGKASAN HASIL (DATASET 15) - BEBERAPA BARIS ===
cabang          sku   variant                exog_used     order seasonal_order  RMSE_train   RMSE_test   LB_p12
   02A  BNOP400CHAR     event               event_flag (2, 1, 1)  (0, 0, 0, 12)  665.621129         NaN 0.962245
   02A  BNOP400CPOX     event               event_flag (2, 1, 1)  (0, 0, 0, 12)  665.621129         NaN 0.962245
   02A  BUVW001K194 event_hol event_flag,holiday_count (2, 0, 1)  (0, 0, 0, 12)  383.261847         NaN 0.338005
   02A   BUVW001KSB event_hol event_flag,holiday_count (2, 0, 1)  (0, 0, 0, 12)  547.539115         NaN 0.815757
   02A  BUVW001KSBM      none                   (none) (2, 0, 1)  (0, 0, 0, 12)  348.688982         NaN 0.841516
   02A   BUVW001KSW event_hol event_flag,holiday_count (1, 1, 1)  (0, 0, 0, 12) 1724.321914 2071.670881 0.364555
   02A  BUVW100C192      none                   (none) (2, 0, 1)  (0, 0, 0, 12)  414.259315         NaN 0.254987
   02A   BUVW

In [5]:
# =========================================================
# SARIMAX BASELINE (DATASET 15) - LOG1P(QTY)
# Pemilihan model:
#   - HANYA pakai RMSE_train (utama) + AIC_train (tie-break)
#   - Test metrics (RMSE_test, MSE_test, dst) murni untuk evaluasi
#
# Kombinasi exog yang dicoba:
#   (1) none
#   (2) event_flag
#   (3) event_flag + holiday_count
#   (4) event_flag + holiday_count + rainfall_lag1
#
# d_suggest diambil dari model_profiles_selected15.csv
# Target SELALU log1p(qty), evaluasi di level qty
# =========================================================
import itertools
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import acorr_ljungbox

warnings.filterwarnings("ignore")

# ---------------- PATH ----------------
PROJECT_ROOT = Path(r"D:\Documents\Skripsi\demand-forecasting")

DATA_PATH    = PROJECT_ROOT / "data" / "dataset_15" / "sarimax_selected15_series.csv"
PROFILE_PATH = PROJECT_ROOT / "data" / "dataset_15" / "model_profiles_selected15.csv"

OUT_DIR   = PROJECT_ROOT / "outputs" / "sarimax_selected15_log_trainselect"
CHART_DIR = OUT_DIR / "charts"
OUT_DIR.mkdir(parents=True, exist_ok=True)
CHART_DIR.mkdir(parents=True, exist_ok=True)

# ---------------- LOAD ----------------
df = pd.read_csv(DATA_PATH, parse_dates=["periode"]).sort_values(
    ["cabang", "sku", "periode"]
)

profiles = pd.read_csv(PROFILE_PATH)

# map d_suggest per (cabang, sku) dari model_profiles
if "sarimax_d_sug" in profiles.columns:
    d_suggest_map = dict(
        zip(
            zip(profiles["cabang"], profiles["sku"]),
            profiles["sarimax_d_sug"].astype(int),
        )
    )
else:
    print("[WARN] Kolom sarimax_d_sug tidak ada di model_profiles. Pakai d=1 semua.")
    d_suggest_map = {}

# ---------------- QUICK SANITY ----------------
expected_cols = [
    "cabang",
    "sku",
    "periode",
    "qty",
    "event_flag",
    "is_train",
    "is_test",
]
missing = [c for c in expected_cols if c not in df.columns]
if missing:
    raise ValueError(f"Kolom wajib hilang: {missing}")

dups = df.duplicated(subset=["cabang", "sku", "periode"]).sum()
if dups:
    raise ValueError(f"Duplikat (cabang, sku, periode): {dups} baris")

test_total = int((df["is_test"] == 1).sum())
print(f"[INFO] Total baris TEST: {test_total}")

for c in ["qty", "event_flag", "holiday_count", "rainfall_lag1"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# ---------------- METRICS ----------------
def mse(a, f):
    a, f = np.asarray(a, float), np.asarray(f, float)
    return float(np.mean((a - f) ** 2))

def mape(a, f):
    a, f = np.asarray(a, float), np.asarray(f, float)
    mask = a != 0
    return float(np.mean(np.abs((a[mask] - f[mask]) / a[mask])) * 100) if mask.any() else np.nan

def smape(a, f):
    a, f = np.asarray(a, float), np.asarray(f, float)
    denom = (np.abs(a) + np.abs(f)) / 2.0
    mask = denom != 0
    return float(np.mean(np.abs(a[mask] - f[mask]) / denom[mask]) * 100) if mask.any() else np.nan

# ---------------- PARAM GRID ----------------
def param_grid_small(d_fix=None):
    """
    Grid kecil untuk (p,d,q) dan (P,D,Q,12).
    Kalau d_fix tidak None, pakai hanya d_fix.
    """
    yielded = set()
    d_values = [0, 1] if d_fix is None else [int(d_fix)]

    for p, d, q in itertools.product([0, 1, 2], d_values, [0, 1, 2]):
        for P, D, Q in itertools.product([0, 1], [0, 1], [0, 1]):
            if (p + q + P + Q) <= 6:
                od, sod = (p, d, q), (P, D, Q, 12)
                if (od, sod) in yielded:
                    continue
                yielded.add((od, sod))
                yield od, sod

# ---------------- FIT 1 KANDIDAT (SELALU LOG1P) ----------------
def fit_eval_log(y_tr, X_tr, y_te, X_te, order, sorder):
    """
    Train SARIMAX dengan target log1p(y), evaluasi di level qty.
    """
    # transform target
    ytr_log = np.log1p(np.clip(y_tr, a_min=0, a_max=None))

    model = SARIMAX(
        ytr_log,
        exog=X_tr,
        order=order,
        seasonal_order=sorder,
        enforce_stationarity=False,
        enforce_invertibility=False,
    )
    res = model.fit(disp=False)

    # fitted (train) di log, balik ke level
    fitted_tr_log = res.fittedvalues
    n_tr = min(len(y_tr), len(fitted_tr_log))
    y_tr_cut = y_tr[-n_tr:]
    fitted_tr_log = np.asarray(fitted_tr_log[-n_tr:], float)
    fitted_tr = np.expm1(fitted_tr_log)

    rmse_train = float(np.sqrt(np.mean((y_tr_cut - fitted_tr) ** 2)))
    mse_train  = mse(y_tr_cut, fitted_tr)
    mae_train  = float(np.mean(np.abs(y_tr_cut - fitted_tr)))
    mape_train = mape(y_tr_cut, fitted_tr)
    smape_train = smape(y_tr_cut, fitted_tr)

    # forecast test (hanya untuk evaluasi, tidak untuk seleksi)
    if y_te is not None and len(y_te) > 0:
        fh = len(y_te)
        pred_log = res.get_forecast(steps=fh, exog=X_te).predicted_mean
        yhat = np.expm1(pred_log)

        rmse_test = float(np.sqrt(np.mean((y_te - yhat) ** 2)))
        mse_test  = mse(y_te, yhat)
        mae_test  = float(np.mean(np.abs(y_te - yhat)))
        mape_test = mape(y_te, yhat)
        smape_test = smape(y_te, yhat)
    else:
        yhat = np.array([], float)
        rmse_test = np.nan
        mse_test  = np.nan
        mae_test  = np.nan
        mape_test = np.nan
        smape_test = np.nan

    resid = res.resid
    try:
        lb_p = float(
            acorr_ljungbox(resid, lags=[12], return_df=True)["lb_pvalue"].iloc[-1]
        )
    except Exception:
        lb_p = np.nan

    return dict(
        order=order,
        seasonal_order=sorder,
        AIC_train=float(res.aic),

        RMSE_train=rmse_train,
        MSE_train=mse_train,
        MAE_train=mae_train,
        MAPE_train=mape_train,
        sMAPE_train=smape_train,

        RMSE_test=rmse_test,
        MSE_test=mse_test,
        MAE_test=mae_test,
        MAPE_test=mape_test,
        sMAPE_test=smape_test,

        LB_p12=lb_p,
        yhat=yhat,
    )

# ---------------- GRID 1 SERI + 1 VARIAN EXOG ----------------
def grid_fit_series_log(g, exog_cols=None, d_fix=None):
    """
    Grid search untuk 1 seri (cabang, sku) dan 1 set exog_cols tertentu.
    Target selalu log1p.

    PEMILIHAN MODEL:
      - Utama: RMSE_train (lebih kecil lebih baik)
      - Tie-break: AIC_train (lebih kecil lebih baik)
    """
    g = g.sort_values("periode").copy()
    y = g["qty"].astype(float).values
    X = g[exog_cols].astype(float).values if exog_cols else None

    m_tr = g["is_train"].astype(bool).values
    m_te = g["is_test"].astype(bool).values

    y_tr = y[m_tr]
    y_te = y[m_te]
    X_tr = X[m_tr] if X is not None else None
    X_te = X[m_te] if X is not None else None

    # bersihkan NaN di train
    if X_tr is not None:
        ok = ~np.isnan(y_tr)
        y_tr, X_tr = y_tr[ok], X_tr[ok]
        ok = ~np.isnan(X_tr).any(axis=1)
        y_tr, X_tr = y_tr[ok], X_tr[ok]
    else:
        y_tr = y_tr[~np.isnan(y_tr)]

    if len(y_tr) < 24:
        return None, pd.DataFrame()

    # bersihkan test exog kalau ada (cuma untuk evaluasi)
    if X_te is not None and len(X_te) > 0:
        ok_te = ~np.isnan(X_te).any(axis=1)
        y_te = y_te[ok_te]
        X_te = X_te[ok_te]

    tried, best = [], None
    for od, sod in param_grid_small(d_fix=d_fix):
        try:
            rec = fit_eval_log(y_tr, X_tr, y_te, X_te, od, sod)
            tried.append(
                {
                    k: rec[k]
                    for k in [
                        "order",
                        "seasonal_order",
                        "AIC_train",
                        "RMSE_train",
                        "MSE_train",
                        "MAE_train",
                        "MAPE_train",
                        "sMAPE_train",
                        "RMSE_test",
                        "MSE_test",
                        "MAE_test",
                        "MAPE_test",
                        "sMAPE_test",
                        "LB_p12",
                    ]
                }
            )

            # SELEKSI model: hanya pakai RMSE_train + AIC_train
            if best is None:
                best = rec
            else:
                better = (
                    (rec["RMSE_train"] < best["RMSE_train"]) or
                    (
                        rec["RMSE_train"] == best["RMSE_train"]
                        and rec["AIC_train"] < best["AIC_train"]
                    )
                )
                if better:
                    best = rec

        except Exception:
            continue

    tried_df = (
        pd.DataFrame(tried)
        .sort_values(["RMSE_train", "AIC_train"])
        .reset_index(drop=True)
        if tried
        else pd.DataFrame()
    )
    return best, tried_df

# ---------------- PLOT TEST ----------------
def plot_test_series(preds_df, cabang, sku, out_dir: Path):
    d = preds_df[(preds_df["cabang"] == cabang) & (preds_df["sku"] == sku)].copy()
    if d.empty:
        return
    d = d.sort_values("periode")
    d["periode"] = pd.to_datetime(d["periode"])
    rmse_val = float(np.sqrt(np.mean((d["qty"].values - d["pred"].values) ** 2)))

    fig, ax = plt.subplots(figsize=(10.5, 3.6))
    locator = mdates.MonthLocator(interval=1)
    ax.xaxis.set_major_locator(locator)
    ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(locator))
    ax.plot(d["periode"], d["qty"], lw=1.8, marker="o", label="Actual")
    ax.plot(d["periode"], d["pred"], lw=1.8, marker="s", label="Pred")
    ax.set_title(f"{cabang}-{sku} | RMSE_test={rmse_val:.1f}")
    ax.set_xlabel("Periode")
    ax.set_ylabel("Qty")
    ax.grid(True, axis="y", alpha=0.25)
    ax.legend(loc="center left", bbox_to_anchor=(1.02, 0.5), frameon=False)
    fig.tight_layout(rect=[0, 0, 0.82, 1])
    fpath = out_dir / f"test_plot__{cabang}__{sku}.png"
    fig.savefig(fpath, dpi=160)
    plt.close(fig)

# ---------------- LOOP PER SERI ----------------
records, preds = [], []

pairs = (
    df[["cabang", "sku"]]
    .drop_duplicates()
    .itertuples(index=False, name=None)
)

for cab, sku in pairs:
    g = df[(df["cabang"] == cab) & (df["sku"] == sku)].copy()

    d_val = int(d_suggest_map.get((cab, sku), 1))

    exog_candidates = []
    exog_candidates.append(("none", []))
    if "event_flag" in g.columns:
        exog_candidates.append(("event", ["event_flag"]))
    if {"event_flag", "holiday_count"}.issubset(g.columns):
        exog_candidates.append(("event_hol", ["event_flag", "holiday_count"]))
    if {"event_flag", "holiday_count", "rainfall_lag1"}.issubset(g.columns):
        exog_candidates.append(
            (
                "event_hol_rain",
                ["event_flag", "holiday_count", "rainfall_lag1"],
            )
        )

    best_tag = None
    best_rec = None
    best_exog_cols = []

    # pilih kombinasi exog terbaik pakai RMSE_train + AIC_train
    for tag, cols in exog_candidates:
        b, _ = grid_fit_series_log(
            g,
            exog_cols=cols if cols else None,
            d_fix=d_val,
        )
        if not b:
            continue

        if best_rec is None:
            best_tag = tag
            best_rec = b
            best_exog_cols = cols
        else:
            better = (
                (b["RMSE_train"] < best_rec["RMSE_train"]) or
                (
                    b["RMSE_train"] == best_rec["RMSE_train"]
                    and b["AIC_train"] < best_rec["AIC_train"]
                )
            )
            if better:
                best_tag = tag
                best_rec = b
                best_exog_cols = cols

    if best_rec is None:
        print(f"[SKIP] {cab}-{sku}: gagal fit (train terlalu pendek atau error).")
        continue

    records.append(
        {
            "cabang": cab,
            "sku": sku,
            "variant": best_tag + "+log",  # penanda bahwa target log1p
            "exog_used": ",".join(best_exog_cols) if best_exog_cols else "(none)",

            "order": best_rec["order"],
            "seasonal_order": best_rec["seasonal_order"],
            "AIC_train": best_rec["AIC_train"],

            "RMSE_train": best_rec["RMSE_train"],
            "MSE_train": best_rec["MSE_train"],
            "MAE_train": best_rec["MAE_train"],
            "MAPE%_train": best_rec["MAPE_train"],
            "sMAPE%_train": best_rec["sMAPE_train"],

            "RMSE_test": best_rec["RMSE_test"],
            "MSE_test": best_rec["MSE_test"],
            "MAE_test": best_rec["MAE_test"],
            "MAPE%_test": best_rec["MAPE_test"],
            "sMAPE%_test": best_rec["sMAPE_test"],

            "LB_p12": best_rec["LB_p12"],
        }
    )

    # simpan pred test kalau ada
    m_te = g["is_test"] == 1
    if m_te.any() and len(best_rec["yhat"]) > 0:
        preds.append(
            pd.DataFrame(
                {
                    "periode": g.loc[m_te, "periode"].values,
                    "qty": g.loc[m_te, "qty"].values,
                    "pred": np.asarray(best_rec["yhat"], float),
                    "cabang": cab,
                    "sku": sku,
                }
            )
        )

# ---------------- OUTPUT & SAVE ----------------
results_df = (
    pd.DataFrame(records)
    .sort_values(["cabang", "sku"])
    .reset_index(drop=True)
)
forecasts_df = (
    pd.concat(preds, ignore_index=True) if len(preds) else pd.DataFrame()
)

print("\n=== RINGKASAN HASIL (DATASET 15, LOG1P TARGET, TRAIN-ONLY SELECTION) - BEBERAPA BARIS ===")
if not results_df.empty:
    print(
        results_df[
            [
                "cabang",
                "sku",
                "variant",
                "exog_used",
                "RMSE_train",
                "MSE_train",
                "MAE_train",
                "MAPE%_train",
                "sMAPE%_train",
                "RMSE_test",
                "MSE_test",
                "MAE_test",
                "MAPE%_test",
                "sMAPE%_test",
                "LB_p12",
            ]
        ].head(20).to_string(index=False)
    )
else:
    print("Tidak ada hasil. Seri terlalu pendek atau split bermasalah.")

METRICS_PATH = OUT_DIR / "sarimax_selected15_log_trainselect_metrics.csv"
PREDS_PATH   = OUT_DIR / "sarimax_selected15_log_trainselect_predictions.csv"
results_df.to_csv(METRICS_PATH, index=False)
forecasts_df.to_csv(PREDS_PATH, index=False)
print(f"\nSaved metrics  -> {METRICS_PATH}")
print(f"Saved preds    -> {PREDS_PATH}")

# CHARTS (hanya seri dengan test)
if not forecasts_df.empty:
    for cab, sk in (
        forecasts_df[["cabang", "sku"]].drop_duplicates().itertuples(
            index=False, name=None
        )
    ):
        plot_test_series(forecasts_df, cab, sk, CHART_DIR)
    print(f"Saved charts   -> {CHART_DIR}")
else:
    print("Tidak ada prediksi test untuk digambar.")


[INFO] Total baris TEST: 45

=== RINGKASAN HASIL (DATASET 15, LOG1P TARGET, TRAIN-ONLY SELECTION) - BEBERAPA BARIS ===
cabang          sku       variant                exog_used  RMSE_train    MSE_train   MAE_train  MAPE%_train  sMAPE%_train   RMSE_test     MSE_test    MAE_test  MAPE%_test  sMAPE%_test   LB_p12
   02A  BNOP400CHAR     event+log               event_flag  626.648831 3.926888e+05  477.690362   106.334518     80.326581         NaN          NaN         NaN         NaN          NaN 0.808845
   02A  BNOP400CPOX     event+log               event_flag  626.648831 3.926888e+05  477.690362   106.334518     80.326581         NaN          NaN         NaN         NaN          NaN 0.808845
   02A  BUVW001K194 event_hol+log event_flag,holiday_count  367.585462 1.351191e+05  287.621412    42.624603     42.670006         NaN          NaN         NaN         NaN          NaN 0.998066
   02A   BUVW001KSB     event+log               event_flag  526.190981 2.768769e+05  413.571270    42.185

In [6]:
results_df["gap_RMSE"] = results_df["RMSE_test"] - results_df["RMSE_train"]
results_df["ratio_RMSE"] = results_df["RMSE_test"] / results_df["RMSE_train"]

results_df[["cabang","sku","RMSE_train","RMSE_test","gap_RMSE","ratio_RMSE"]].head(20)


,cabang,sku,RMSE_train,RMSE_test,gap_RMSE,ratio_RMSE
0,02A,BNOP400CHAR,626.648831,NaN,NaN,NaN
1,02A,BNOP400CPOX,626.648831,NaN,NaN,NaN
2,02A,BUVW001K194,367.585462,NaN,NaN,NaN
3,02A,BUVW001KSB,526.190981,NaN,NaN,NaN
4,02A,BUVW001KSBM,344.749376,NaN,NaN,NaN
5,02A,BUVW001KSW,1732.555022,1959.400426,226.845404,1.130931
6,02A,BUVW100C192,412.246511,NaN,NaN,NaN
7,02A,BUVW100CSB,827.895709,NaN,NaN,NaN
8,02A,BUVW100CSW,852.590211,NaN,NaN,NaN
9,02A,CKLM001KS607,407.764228,NaN,NaN,NaN


ini sdh sesuai eda

In [ ]:
# =========================================================
# SARIMAX BASELINE (DATASET 15) - LOG1P(QTY) - INI YANG DIPAKE ***
# - Dataset: panel_exog_selected15.csv
# - Target: log1p(qty), evaluasi di level qty
# - Seleksi model:
#       Utama  : RMSE_train (lebih kecil lebih baik)
#       Tie-break: AIC_train (lebih kecil lebih baik)
# - Kombinasi exog:
#       (1) none
#       (2) event_flag
#       (3) event_flag + holiday_count
#       (4) event_flag + holiday_count + rainfall_lag1 (kalau ada)
# - Tidak ada data test dipakai di training / seleksi
# =========================================================
import itertools
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import acorr_ljungbox

warnings.filterwarnings("ignore")

# ---------------- PATH ----------------
PROJECT_ROOT = Path(r"D:\Documents\Skripsi\demand-forecasting")

DATA_PATH    = PROJECT_ROOT / "data" / "dataset_15" / "panel_exog_selected15.csv"
PROFILE_PATH = PROJECT_ROOT / "data" / "dataset_15" / "model_profiles_selected15.csv"

OUT_DIR   = PROJECT_ROOT / "outputs" / "sarimax_15_log_trainselect"
CHART_DIR = OUT_DIR / "charts"
OUT_DIR.mkdir(parents=True, exist_ok=True)
CHART_DIR.mkdir(parents=True, exist_ok=True)

# ---------------- LOAD ----------------
print("Load panel:", DATA_PATH)
df = (
    pd.read_csv(DATA_PATH, parse_dates=["periode"])
      .sort_values(["cabang", "sku", "periode"])
)

profiles = pd.read_csv(PROFILE_PATH)

# map d_suggest per (cabang, sku) dari model_profiles
if "sarimax_d_sug" in profiles.columns:
    d_suggest_map = dict(
        zip(
            zip(profiles["cabang"], profiles["sku"]),
            profiles["sarimax_d_sug"].astype(int),
        )
    )
else:
    print("[WARN] Kolom sarimax_d_sug tidak ada di model_profiles. Pakai d=1 semua.")
    d_suggest_map = {}

# ---------------- QUICK SANITY ----------------
expected_cols = [
    "cabang",
    "sku",
    "periode",
    "qty",
    "event_flag",
    "holiday_count",
    "rainfall_lag1",
    "is_train",
    "is_test",
]
missing = [c for c in expected_cols if c not in df.columns]
if missing:
    raise ValueError(f"Kolom wajib hilang: {missing}")

dups = df.duplicated(subset=["cabang", "sku", "periode"]).sum()
if dups:
    raise ValueError(f"Duplikat (cabang, sku, periode): {dups} baris")

test_total = int((df["is_test"] == 1).sum())
print(f"[INFO] Total baris TEST: {test_total}")

for c in ["qty", "event_flag", "holiday_count", "rainfall_lag1"]:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# ---------------- METRICS ----------------
def mse(a, f):
    a, f = np.asarray(a, float), np.asarray(f, float)
    return float(np.mean((a - f) ** 2))

def mape(a, f):
    a, f = np.asarray(a, float), np.asarray(f, float)
    mask = a != 0
    return float(np.mean(np.abs((a[mask] - f[mask]) / a[mask])) * 100) if mask.any() else np.nan

def smape(a, f):
    a, f = np.asarray(a, float), np.asarray(f, float)
    denom = (np.abs(a) + np.abs(f)) / 2.0
    mask = denom != 0
    return float(np.mean(np.abs(a[mask] - f[mask]) / denom[mask]) * 100) if mask.any() else np.nan

# ---------------- PARAM GRID ----------------
def param_grid_small(d_fix=None):
    """
    Grid kecil untuk (p,d,q) dan (P,D,Q,12).
    Kalau d_fix tidak None, pakai hanya d_fix.
    """
    yielded = set()
    d_values = [0, 1] if d_fix is None else [int(d_fix)]

    for p, d, q in itertools.product([0, 1, 2], d_values, [0, 1, 2]):
        for P, D, Q in itertools.product([0, 1], [0, 1], [0, 1]):
            # batasi kompleksitas sedikit supaya nggak barbar
            if (p + q + P + Q) <= 6:
                od, sod = (p, d, q), (P, D, Q, 12)
                if (od, sod) in yielded:
                    continue
                yielded.add((od, sod))
                yield od, sod

# ---------------- FIT 1 KANDIDAT (SELALU LOG1P) ----------------
def fit_eval_log(y_tr, X_tr, y_te, X_te, order, sorder):
    """
    Train SARIMAX dengan target log1p(y), evaluasi di level qty.
    Seleksi pakai RMSE_train + AIC_train, test cuma buat laporan.
    """
    # transform target
    ytr_log = np.log1p(np.clip(y_tr, a_min=0, a_max=None))

    model = SARIMAX(
        ytr_log,
        exog=X_tr,
        order=order,
        seasonal_order=sorder,
        enforce_stationarity=False,
        enforce_invertibility=False,
    )
    res = model.fit(disp=False)

    # fitted (train) di log, balik ke level
    fitted_tr_log = res.fittedvalues
    n_tr = min(len(y_tr), len(fitted_tr_log))
    y_tr_cut = y_tr[-n_tr:]
    fitted_tr_log = np.asarray(fitted_tr_log[-n_tr:], float)
    fitted_tr = np.expm1(fitted_tr_log)

    rmse_train = float(np.sqrt(np.mean((y_tr_cut - fitted_tr) ** 2)))
    mse_train  = mse(y_tr_cut, fitted_tr)
    mae_train  = float(np.mean(np.abs(y_tr_cut - fitted_tr)))
    mape_train = mape(y_tr_cut, fitted_tr)
    smape_train = smape(y_tr_cut, fitted_tr)

    # forecast test (hanya evaluasi, tidak mempengaruhi seleksi)
    if y_te is not None and len(y_te) > 0:
        fh = len(y_te)
        pred_log = res.get_forecast(steps=fh, exog=X_te).predicted_mean
        yhat = np.expm1(pred_log)

        rmse_test = float(np.sqrt(np.mean((y_te - yhat) ** 2)))
        mse_test  = mse(y_te, yhat)
        mae_test  = float(np.mean(np.abs(y_te - yhat)))
        mape_test = mape(y_te, yhat)
        smape_test = smape(y_te, yhat)
    else:
        yhat = np.array([], float)
        rmse_test = np.nan
        mse_test  = np.nan
        mae_test  = np.nan
        mape_test = np.nan
        smape_test = np.nan

    resid = res.resid
    try:
        lb_p = float(
            acorr_ljungbox(resid, lags=[12], return_df=True)["lb_pvalue"].iloc[-1]
        )
    except Exception:
        lb_p = np.nan

    return dict(
        order=order,
        seasonal_order=sorder,
        AIC_train=float(res.aic),

        RMSE_train=rmse_train,
        MSE_train=mse_train,
        MAE_train=mae_train,
        MAPE_train=mape_train,
        sMAPE_train=smape_train,

        RMSE_test=rmse_test,
        MSE_test=mse_test,
        MAE_test=mae_test,
        MAPE_test=mape_test,
        sMAPE_test=smape_test,

        LB_p12=lb_p,
        yhat=yhat,
    )

# ---------------- GRID 1 SERI + 1 VARIAN EXOG ----------------
def grid_fit_series_log(g, exog_cols=None, d_fix=None):
    """
    Grid search untuk 1 seri (cabang, sku) dan 1 set exog_cols tertentu.
    Target selalu log1p.

    PEMILIHAN MODEL:
      - Utama: RMSE_train (lebih kecil lebih baik)
      - Tie-break: AIC_train (lebih kecil lebih baik)
    """
    g = g.sort_values("periode").copy()
    y = g["qty"].astype(float).values
    X = g[exog_cols].astype(float).values if exog_cols else None

    m_tr = g["is_train"].astype(bool).values
    m_te = g["is_test"].astype(bool).values

    y_tr = y[m_tr]
    y_te = y[m_te]
    X_tr = X[m_tr] if X is not None else None
    X_te = X[m_te] if X is not None else None

    # bersihkan NaN di train
    if X_tr is not None:
        ok = ~np.isnan(y_tr)
        y_tr, X_tr = y_tr[ok], X_tr[ok]
        ok = ~np.isnan(X_tr).any(axis=1)
        y_tr, X_tr = y_tr[ok], X_tr[ok]
    else:
        y_tr = y_tr[~np.isnan(y_tr)]

    if len(y_tr) < 24:
        return None, pd.DataFrame()

    # bersihkan test exog kalau ada (cuma untuk evaluasi)
    if X_te is not None and len(X_te) > 0:
        ok_te = ~np.isnan(X_te).any(axis=1)
        y_te = y_te[ok_te]
        X_te = X_te[ok_te]

    tried, best = [], None
    for od, sod in param_grid_small(d_fix=d_fix):
        try:
            rec = fit_eval_log(y_tr, X_tr, y_te, X_te, od, sod)
            tried.append(
                {
                    k: rec[k]
                    for k in [
                        "order",
                        "seasonal_order",
                        "AIC_train",
                        "RMSE_train",
                        "MSE_train",
                        "MAE_train",
                        "MAPE_train",
                        "sMAPE_train",
                        "RMSE_test",
                        "MSE_test",
                        "MAE_test",
                        "MAPE_test",
                        "sMAPE_test",
                        "LB_p12",
                    ]
                }
            )

            # SELEKSI model: hanya pakai RMSE_train + AIC_train
            if best is None:
                best = rec
            else:
                better = (
                    (rec["RMSE_train"] < best["RMSE_train"]) or
                    (
                        rec["RMSE_train"] == best["RMSE_train"]
                        and rec["AIC_train"] < best["AIC_train"]
                    )
                )
                if better:
                    best = rec

        except Exception:
            continue

    tried_df = (
        pd.DataFrame(tried)
        .sort_values(["RMSE_train", "AIC_train"])
        .reset_index(drop=True)
        if tried
        else pd.DataFrame()
    )
    return best, tried_df

# ---------------- PLOT TEST ----------------
def plot_test_series(preds_df, cabang, sku, out_dir: Path):
    d = preds_df[(preds_df["cabang"] == cabang) & (preds_df["sku"] == sku)].copy()
    if d.empty:
        return
    d = d.sort_values("periode")
    d["periode"] = pd.to_datetime(d["periode"])
    rmse_val = float(np.sqrt(np.mean((d["qty"].values - d["pred"].values) ** 2)))

    fig, ax = plt.subplots(figsize=(10.5, 3.6))
    locator = mdates.MonthLocator(interval=1)
    ax.xaxis.set_major_locator(locator)
    ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(locator))
    ax.plot(d["periode"], d["qty"], lw=1.8, marker="o", label="Actual")
    ax.plot(d["periode"], d["pred"], lw=1.8, marker="s", label="Pred")
    ax.set_title(f"{cabang}-{sku} | RMSE_test={rmse_val:.1f}")
    ax.set_xlabel("Periode")
    ax.set_ylabel("Qty")
    ax.grid(True, axis="y", alpha=0.25)
    ax.legend(loc="center left", bbox_to_anchor=(1.02, 0.5), frameon=False)
    fig.tight_layout(rect=[0, 0, 0.82, 1])
    fpath = out_dir / f"test_plot__{cabang}__{sku}.png"
    fig.savefig(fpath, dpi=160)
    plt.close(fig)

# ---------------- LOOP PER SERI ----------------
records, preds = [], []

pairs = (
    df[["cabang", "sku"]]
    .drop_duplicates()
    .itertuples(index=False, name=None)
)

for cab, sku in pairs:
    g = df[(df["cabang"] == cab) & (df["sku"] == sku)].copy()

    d_val = int(d_suggest_map.get((cab, sku), 1))

    # kandidat exog persis sesuai spek
    exog_candidates = []
    exog_candidates.append(("none", []))
    exog_candidates.append(("event", ["event_flag"]))
    exog_candidates.append(("event_hol", ["event_flag", "holiday_count"]))
    exog_candidates.append(
        ("event_hol_rain", ["event_flag", "holiday_count", "rainfall_lag1"])
    )

    best_tag = None
    best_rec = None
    best_exog_cols = []

    # pilih kombinasi exog terbaik pakai RMSE_train + AIC_train
    for tag, cols in exog_candidates:
        # pastikan kolomnya memang ada
        if cols and not set(cols).issubset(g.columns):
            continue

        b, _ = grid_fit_series_log(
            g,
            exog_cols=cols if cols else None,
            d_fix=d_val,
        )
        if not b:
            continue

        if best_rec is None:
            best_tag = tag
            best_rec = b
            best_exog_cols = cols
        else:
            better = (
                (b["RMSE_train"] < best_rec["RMSE_train"]) or
                (
                    b["RMSE_train"] == best_rec["RMSE_train"]
                    and b["AIC_train"] < best_rec["AIC_train"]
                )
            )
            if better:
                best_tag = tag
                best_rec = b
                best_exog_cols = cols

    if best_rec is None:
        print(f"[SKIP] {cab}-{sku}: gagal fit (train terlalu pendek atau error).")
        continue

    records.append(
        {
            "cabang": cab,
            "sku": sku,
            "variant": best_tag + "+log",  # penanda bahwa target log1p
            "exog_used": ",".join(best_exog_cols) if best_exog_cols else "(none)",

            "order": best_rec["order"],
            "seasonal_order": best_rec["seasonal_order"],
            "AIC_train": best_rec["AIC_train"],

            "RMSE_train": best_rec["RMSE_train"],
            "MSE_train": best_rec["MSE_train"],
            "MAE_train": best_rec["MAE_train"],
            "MAPE%_train": best_rec["MAPE_train"],
            "sMAPE%_train": best_rec["sMAPE_train"],

            "RMSE_test": best_rec["RMSE_test"],
            "MSE_test": best_rec["MSE_test"],
            "MAE_test": best_rec["MAE_test"],
            "MAPE%_test": best_rec["MAPE_test"],
            "sMAPE%_test": best_rec["sMAPE_test"],

            "LB_p12": best_rec["LB_p12"],
        }
    )

    # simpan pred test kalau ada
    m_te = g["is_test"] == 1
    if m_te.any() and len(best_rec["yhat"]) > 0:
        preds.append(
            pd.DataFrame(
                {
                    "periode": g.loc[m_te, "periode"].values,
                    "qty": g.loc[m_te, "qty"].values,
                    "pred": np.asarray(best_rec["yhat"], float),
                    "cabang": cab,
                    "sku": sku,
                }
            )
        )

# ---------------- OUTPUT & SAVE ----------------
results_df = (
    pd.DataFrame(records)
    .sort_values(["cabang", "sku"])
    .reset_index(drop=True)
)
forecasts_df = (
    pd.concat(preds, ignore_index=True) if len(preds) else pd.DataFrame()
)

print("\n=== RINGKASAN HASIL (DATASET 15, LOG1P TARGET, TRAIN-ONLY SELECTION) - BEBERAPA BARIS ===")
if not results_df.empty:
    print(
        results_df[
            [
                "cabang",
                "sku",
                "variant",
                "exog_used",
                "RMSE_train",
                "MSE_train",
                "MAE_train",
                "MAPE%_train",
                "sMAPE%_train",
                "RMSE_test",
                "MSE_test",
                "MAE_test",
                "MAPE%_test",
                "sMAPE%_test",
                "LB_p12",
            ]
        ].head(20).to_string(index=False)
    )
else:
    print("Tidak ada hasil. Seri terlalu pendek atau split bermasalah.")

METRICS_PATH = OUT_DIR / "sarimax_15_log_trainselect_metrics.csv"
PREDS_PATH   = OUT_DIR / "sarimax_15_log_trainselect_predictions.csv"
results_df.to_csv(METRICS_PATH, index=False)
forecasts_df.to_csv(PREDS_PATH, index=False)
print(f"\nSaved metrics  -> {METRICS_PATH}")
print(f"Saved preds    -> {PREDS_PATH}")

# CHARTS (hanya seri dengan test)
if not forecasts_df.empty:
    for cab, sk in (
        forecasts_df[["cabang", "sku"]].drop_duplicates().itertuples(
            index=False, name=None
        )
    ):
        plot_test_series(forecasts_df, cab, sk, CHART_DIR)
    print(f"Saved charts   -> {CHART_DIR}")
else:
    print("Tidak ada prediksi test untuk digambar.")


Load panel: D:\Documents\Skripsi\demand-forecasting\data\dataset_15\panel_exog_selected15.csv
[INFO] Total baris TEST: 45



=== RINGKASAN HASIL (DATASET 15, LOG1P TARGET, TRAIN-ONLY SELECTION) - BEBERAPA BARIS ===
cabang          sku       variant                exog_used  RMSE_train    MSE_train   MAE_train  MAPE%_train  sMAPE%_train   RMSE_test     MSE_test    MAE_test  MAPE%_test  sMAPE%_test   LB_p12
   02A  BNOP400CHAR     event+log               event_flag  617.176356 3.809067e+05  468.957887   104.223180     79.433353         NaN          NaN         NaN         NaN          NaN 0.782529
   02A  BNOP400CPOX     event+log               event_flag  617.176356 3.809067e+05  468.957887   104.223180     79.433353         NaN          NaN         NaN         NaN          NaN 0.782529
   02A  BUVW001K194 event_hol+log event_flag,holiday_count  345.822599 1.195933e+05  275.742308    42.007140     42.059290         NaN          NaN         NaN         NaN          NaN 0.998710
   02A   BUVW001KSB     event+log               event_flag  509.742497 2.598374e+05  398.974691    40.786020     42.292550         Na

ini coba pake qty 

In [21]:
# =========================================================
# SARIMAX BASELINE (DATASET 15) - LEVEL(QTY)
# - Dataset: panel_exog_selected15.csv
# - Target: qty (level), evaluasi di level qty
# - Seleksi model:
#       Utama  : RMSE_train (lebih kecil lebih baik)
#       Tie-break: AIC_train (lebih kecil lebih baik)
# - Kombinasi exog:
#       (1) none
#       (2) event_flag
#       (3) event_flag + holiday_count
#       (4) event_flag + holiday_count + rainfall_lag1
# =========================================================

import itertools
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import acorr_ljungbox

warnings.filterwarnings("ignore")

# ---------------- PATH ----------------
PROJECT_ROOT = Path(r"D:\Documents\Skripsi\demand-forecasting")

DATA_PATH    = PROJECT_ROOT / "data" / "dataset_15" / "panel_exog_selected15.csv"
PROFILE_PATH = PROJECT_ROOT / "data" / "dataset_15" / "model_profiles_selected15.csv"

OUT_DIR   = PROJECT_ROOT / "outputs" / "sarimax_15_level_trainselect"
CHART_DIR = OUT_DIR / "charts"
OUT_DIR.mkdir(parents=True, exist_ok=True)
CHART_DIR.mkdir(parents=True, exist_ok=True)

# ---------------- LOAD ----------------
print("Load panel:", DATA_PATH)
df = (
    pd.read_csv(DATA_PATH, parse_dates=["periode"])
      .sort_values(["cabang", "sku", "periode"])
)

profiles = pd.read_csv(PROFILE_PATH)

# map d_suggest per (cabang, sku) dari model_profiles
if "sarimax_d_sug" in profiles.columns:
    d_suggest_map = dict(
        zip(
            zip(profiles["cabang"], profiles["sku"]),
            profiles["sarimax_d_sug"].astype(int),
        )
    )
else:
    print("[WARN] Kolom sarimax_d_sug tidak ada di model_profiles. Pakai d=1 semua.")
    d_suggest_map = {}

# ---------------- QUICK SANITY ----------------
expected_cols = [
    "cabang",
    "sku",
    "periode",
    "qty",
    "event_flag",
    "holiday_count",
    "rainfall_lag1",
    "is_train",
    "is_test",
]
missing = [c for c in expected_cols if c not in df.columns]
if missing:
    raise ValueError(f"Kolom wajib hilang: {missing}")

dups = df.duplicated(subset=["cabang", "sku", "periode"]).sum()
if dups:
    raise ValueError(f"Duplikat (cabang, sku, periode): {dups} baris")

test_total = int((df["is_test"] == 1).sum())
print(f"[INFO] Total baris TEST: {test_total}")

for c in ["qty", "event_flag", "holiday_count", "rainfall_lag1"]:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# ---------------- METRICS ----------------
def mse(a, f):
    a, f = np.asarray(a, float), np.asarray(f, float)
    return float(np.mean((a - f) ** 2))

def mape(a, f):
    a, f = np.asarray(a, float), np.asarray(f, float)
    mask = a != 0
    return float(np.mean(np.abs((a[mask] - f[mask]) / a[mask])) * 100) if mask.any() else np.nan

def smape(a, f):
    a, f = np.asarray(a, float), np.asarray(f, float)
    denom = (np.abs(a) + np.abs(f)) / 2.0
    mask = denom != 0
    return float(np.mean(np.abs(a[mask] - f[mask]) / denom[mask]) * 100) if mask.any() else np.nan

# ---------------- PARAM GRID ----------------
def param_grid_small(d_fix=None):
    """
    Grid kecil untuk (p,d,q) dan (P,D,Q,12).
    Kalau d_fix tidak None, pakai hanya d_fix.
    """
    yielded = set()
    d_values = [0, 1] if d_fix is None else [int(d_fix)]

    for p, d, q in itertools.product([0, 1, 2], d_values, [0, 1, 2]):
        for P, D, Q in itertools.product([0, 1], [0, 1], [0, 1]):
            if (p + q + P + Q) <= 6:
                od, sod = (p, d, q), (P, D, Q, 12)
                if (od, sod) in yielded:
                    continue
                yielded.add((od, sod))
                yield od, sod

# ---------------- FIT 1 KANDIDAT (LEVEL QTY) ----------------
def fit_eval_level(y_tr, X_tr, y_te, X_te, order, sorder):
    """
    Train SARIMAX dengan target qty level, evaluasi di level qty.
    Seleksi pakai RMSE_train + AIC_train, test cuma buat laporan.
    """
    ytr = np.asarray(y_tr, float)

    model = SARIMAX(
        ytr,
        exog=X_tr,
        order=order,
        seasonal_order=sorder,
        enforce_stationarity=False,
        enforce_invertibility=False,
    )
    res = model.fit(disp=False)

    # fitted train di level
    fitted_tr = np.asarray(res.fittedvalues, float)
    n_tr = min(len(ytr), len(fitted_tr))
    y_tr_cut = ytr[-n_tr:]
    fitted_tr = fitted_tr[-n_tr:]

    rmse_train = float(np.sqrt(np.mean((y_tr_cut - fitted_tr) ** 2)))
    mse_train  = mse(y_tr_cut, fitted_tr)
    mae_train  = float(np.mean(np.abs(y_tr_cut - fitted_tr)))
    mape_train = mape(y_tr_cut, fitted_tr)
    smape_train = smape(y_tr_cut, fitted_tr)

    # forecast test (level)
    if y_te is not None and len(y_te) > 0:
        fh = len(y_te)
        pred = res.get_forecast(steps=fh, exog=X_te).predicted_mean
        yhat = np.asarray(pred, float)

        rmse_test = float(np.sqrt(np.mean((y_te - yhat) ** 2)))
        mse_test  = mse(y_te, yhat)
        mae_test  = float(np.mean(np.abs(y_te - yhat)))
        mape_test = mape(y_te, yhat)
        smape_test = smape(y_te, yhat)
    else:
        yhat = np.array([], float)
        rmse_test = np.nan
        mse_test  = np.nan
        mae_test  = np.nan
        mape_test = np.nan
        smape_test = np.nan

    resid = res.resid
    try:
        lb_p = float(
            acorr_ljungbox(resid, lags=[12], return_df=True)["lb_pvalue"].iloc[-1]
        )
    except Exception:
        lb_p = np.nan

    return dict(
        order=order,
        seasonal_order=sorder,
        AIC_train=float(res.aic),

        RMSE_train=rmse_train,
        MSE_train=mse_train,
        MAE_train=mae_train,
        MAPE_train=mape_train,
        sMAPE_train=smape_train,

        RMSE_test=rmse_test,
        MSE_test=mse_test,
        MAE_test=mae_test,
        MAPE_test=mape_test,
        sMAPE_test=smape_test,

        LB_p12=lb_p,
        yhat=yhat,
    )

# ---------------- GRID 1 SERI + 1 VARIAN EXOG ----------------
def grid_fit_series_level(g, exog_cols=None, d_fix=None):
    """
    Grid search untuk 1 seri (cabang, sku) dan 1 set exog_cols tertentu.
    Target qty level.

    PEMILIHAN MODEL:
      - Utama: RMSE_train (lebih kecil lebih baik)
      - Tie-break: AIC_train (lebih kecil lebih baik)
    """
    g = g.sort_values("periode").copy()
    y = g["qty"].astype(float).values
    X = g[exog_cols].astype(float).values if exog_cols else None

    m_tr = g["is_train"].astype(bool).values
    m_te = g["is_test"].astype(bool).values

    y_tr = y[m_tr]
    y_te = y[m_te]
    X_tr = X[m_tr] if X is not None else None
    X_te = X[m_te] if X is not None else None

    if X_tr is not None:
        ok = ~np.isnan(y_tr)
        y_tr, X_tr = y_tr[ok], X_tr[ok]
        ok = ~np.isnan(X_tr).any(axis=1)
        y_tr, X_tr = y_tr[ok], X_tr[ok]
    else:
        y_tr = y_tr[~np.isnan(y_tr)]

    if len(y_tr) < 24:
        return None, pd.DataFrame()

    if X_te is not None and len(X_te) > 0:
        ok_te = ~np.isnan(X_te).any(axis=1)
        y_te = y_te[ok_te]
        X_te = X_te[ok_te]

    tried, best = [], None

    for od, sod in param_grid_small(d_fix=d_fix):
        try:
            rec = fit_eval_level(y_tr, X_tr, y_te, X_te, od, sod)
            tried.append(
                {
                    k: rec[k]
                    for k in [
                        "order",
                        "seasonal_order",
                        "AIC_train",
                        "RMSE_train",
                        "MSE_train",
                        "MAE_train",
                        "MAPE_train",
                        "sMAPE_train",
                        "RMSE_test",
                        "MSE_test",
                        "MAE_test",
                        "MAPE_test",
                        "sMAPE_test",
                        "LB_p12",
                    ]
                }
            )

            # seleksi model pakai RMSE_train + AIC_train
            if best is None:
                best = rec
            else:
                better = (
                    (rec["RMSE_train"] < best["RMSE_train"]) or
                    (
                        rec["RMSE_train"] == best["RMSE_train"]
                        and rec["AIC_train"] < best["AIC_train"]
                    )
                )
                if better:
                    best = rec

        except Exception:
            continue

    tried_df = (
        pd.DataFrame(tried)
        .sort_values(["RMSE_train", "AIC_train"])
        .reset_index(drop=True)
        if tried
        else pd.DataFrame()
    )
    return best, tried_df

# ---------------- PLOT TEST ----------------
def plot_test_series(preds_df, cabang, sku, out_dir: Path):
    d = preds_df[(preds_df["cabang"] == cabang) & (preds_df["sku"] == sku)].copy()
    if d.empty:
        return
    d = d.sort_values("periode")
    d["periode"] = pd.to_datetime(d["periode"])
    rmse_val = float(np.sqrt(np.mean((d["qty"].values - d["pred"].values) ** 2)))

    fig, ax = plt.subplots(figsize=(10.5, 3.6))
    locator = mdates.MonthLocator(interval=1)
    ax.xaxis.set_major_locator(locator)
    ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(locator))
    ax.plot(d["periode"], d["qty"], lw=1.8, marker="o", label="Actual")
    ax.plot(d["periode"], d["pred"], lw=1.8, marker="s", label="Pred")
    ax.set_title(f"{cabang}-{sku} | RMSE_test={rmse_val:.1f}")
    ax.set_xlabel("Periode")
    ax.set_ylabel("Qty")
    ax.grid(True, axis="y", alpha=0.25)
    ax.legend(loc="center left", bbox_to_anchor=(1.02, 0.5), frameon=False)
    fig.tight_layout(rect=[0, 0, 0.82, 1])
    fpath = out_dir / f"test_plot__{cabang}__{sku}.png"
    fig.savefig(fpath, dpi=160)
    plt.close(fig)

# ---------------- LOOP PER SERI ----------------
records, preds = [], []

pairs = (
    df[["cabang", "sku"]]
    .drop_duplicates()
    .itertuples(index=False, name=None)
)

for cab, sku in pairs:
    g = df[(df["cabang"] == cab) & (df["sku"] == sku)].copy()

    d_val = int(d_suggest_map.get((cab, sku), 1))

    exog_candidates = []
    exog_candidates.append(("none", []))
    exog_candidates.append(("event", ["event_flag"]))
    exog_candidates.append(("event_hol", ["event_flag", "holiday_count"]))
    exog_candidates.append(
        ("event_hol_rain", ["event_flag", "holiday_count", "rainfall_lag1"])
    )

    best_tag = None
    best_rec = None
    best_exog_cols = []

    # pilih kombinasi exog terbaik pakai RMSE_train + AIC_train
    for tag, cols in exog_candidates:
        if cols and not set(cols).issubset(g.columns):
            continue

        b, _ = grid_fit_series_level(
            g,
            exog_cols=cols if cols else None,
            d_fix=d_val,
        )
        if not b:
            continue

        if best_rec is None:
            best_tag = tag
            best_rec = b
            best_exog_cols = cols
        else:
            better = (
                (b["RMSE_train"] < best_rec["RMSE_train"]) or
                (
                    b["RMSE_train"] == best_rec["RMSE_train"]
                    and b["AIC_train"] < best_rec["AIC_train"]
                )
            )
            if better:
                best_tag = tag
                best_rec = b
                best_exog_cols = cols

    if best_rec is None:
        print(f"[SKIP] {cab}-{sku}: gagal fit (train terlalu pendek atau error).")
        continue

    records.append(
        {
            "cabang": cab,
            "sku": sku,
            "variant": best_tag + "+level",
            "exog_used": ",".join(best_exog_cols) if best_exog_cols else "(none)",

            "order": best_rec["order"],
            "seasonal_order": best_rec["seasonal_order"],
            "AIC_train": best_rec["AIC_train"],

            "RMSE_train": best_rec["RMSE_train"],
            "MSE_train": best_rec["MSE_train"],
            "MAE_train": best_rec["MAE_train"],
            "MAPE%_train": best_rec["MAPE_train"],
            "sMAPE%_train": best_rec["sMAPE_train"],

            "RMSE_test": best_rec["RMSE_test"],
            "MSE_test": best_rec["MSE_test"],
            "MAE_test": best_rec["MAE_test"],
            "MAPE%_test": best_rec["MAPE_test"],
            "sMAPE%_test": best_rec["sMAPE_test"],

            "LB_p12": best_rec["LB_p12"],
        }
    )

    m_te = g["is_test"] == 1
    if m_te.any() and len(best_rec["yhat"]) > 0:
        preds.append(
            pd.DataFrame(
                {
                    "periode": g.loc[m_te, "periode"].values,
                    "qty": g.loc[m_te, "qty"].values,
                    "pred": np.asarray(best_rec["yhat"], float),
                    "cabang": cab,
                    "sku": sku,
                }
            )
        )

# ---------------- OUTPUT & SAVE ----------------
results_df = (
    pd.DataFrame(records)
    .sort_values(["cabang", "sku"])
    .reset_index(drop=True)
)
forecasts_df = (
    pd.concat(preds, ignore_index=True) if len(preds) else pd.DataFrame()
)

print("\n=== RINGKASAN HASIL (DATASET 15, LEVEL TARGET, TRAIN-ONLY SELECTION) - BEBERAPA BARIS ===")
if not results_df.empty:
    print(
        results_df[
            [
                "cabang",
                "sku",
                "variant",
                "exog_used",
                "RMSE_train",
                "MSE_train",
                "MAE_train",
                "MAPE%_train",
                "sMAPE%_train",
                "RMSE_test",
                "MSE_test",
                "MAE_test",
                "MAPE%_test",
                "sMAPE%_test",
                "LB_p12",
            ]
        ].head(20).to_string(index=False)
    )
else:
    print("Tidak ada hasil. Seri terlalu pendek atau split bermasalah.")

METRICS_PATH = OUT_DIR / "sarimax_15_level_trainselect_metrics.csv"
PREDS_PATH   = OUT_DIR / "sarimax_15_level_trainselect_predictions.csv"
results_df.to_csv(METRICS_PATH, index=False)
forecasts_df.to_csv(PREDS_PATH, index=False)
print(f"\nSaved metrics  -> {METRICS_PATH}")
print(f"Saved preds    -> {PREDS_PATH}")

if not forecasts_df.empty:
    for cab, sk in (
        forecasts_df[["cabang", "sku"]].drop_duplicates().itertuples(
            index=False, name=None
        )
    ):
        plot_test_series(forecasts_df, cab, sk, CHART_DIR)
    print(f"Saved charts   -> {CHART_DIR}")
else:
    print("Tidak ada prediksi test untuk digambar.")


Load panel: D:\Documents\Skripsi\demand-forecasting\data\dataset_15\panel_exog_selected15.csv
[INFO] Total baris TEST: 45

=== RINGKASAN HASIL (DATASET 15, LEVEL TARGET, TRAIN-ONLY SELECTION) - BEBERAPA BARIS ===
cabang          sku         variant                exog_used  RMSE_train    MSE_train   MAE_train  MAPE%_train  sMAPE%_train   RMSE_test     MSE_test    MAE_test  MAPE%_test  sMAPE%_test   LB_p12
   02A  BNOP400CHAR     event+level               event_flag  654.381503 4.282152e+05  459.514656   116.739384     69.970671         NaN          NaN         NaN         NaN          NaN 0.971238
   02A  BNOP400CPOX     event+level               event_flag  654.381503 4.282152e+05  459.514656   116.739384     69.970671         NaN          NaN         NaN         NaN          NaN 0.971238
   02A  BUVW001K194     event+level               event_flag  344.350856 1.185775e+05  279.314674    45.183665     42.549927         NaN          NaN         NaN         NaN          NaN 0.706695
   

In [14]:
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tools.sm_exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

PROJECT_ROOT  = Path(r"D:\Documents\Skripsi\demand-forecasting")
DATASET15_DIR = PROJECT_ROOT / "data" / "dataset_15"
OUT_DIR       = PROJECT_ROOT / "outputs" / "sarimax_15_gridsearch_final"

OUT_DIR.mkdir(parents=True, exist_ok=True)
PLOT_DIR = OUT_DIR / "plots_pred"
PLOT_DIR.mkdir(parents=True, exist_ok=True)

PANEL_PATH   = DATASET15_DIR / "panel_exog_selected15.csv"
PROFILE_PATH = DATASET15_DIR / "model_profiles_selected15.csv"

panel = pd.read_csv(PANEL_PATH, parse_dates=["periode"])
profiles = pd.read_csv(PROFILE_PATH)

print("Rows panel 15 :", len(panel))
print("Rows profiles :", len(profiles))

panel = panel.sort_values(["cabang", "sku", "periode"]).reset_index(drop=True)

if "rainfall_lag1" in panel.columns:
    mask_nan = panel["rainfall_lag1"].isna()
    print("NaN rainfall_lag1:", mask_nan.sum())
    panel.loc[mask_nan, "rainfall_lag1"] = 0.0

test_info = (
    panel.groupby(["cabang", "sku"], as_index=False)["is_test"]
         .sum()
         .rename(columns={"is_test": "n_test"})
)
eval9 = test_info.query("n_test > 0").copy()
print("Series with test rows:", len(eval9))
eval9.to_csv(OUT_DIR / "sarimax_eval9_series.csv", index=False)


def mae(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    return np.mean(np.abs(y_true - y_pred))


def mse(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    return np.mean((y_true - y_pred) ** 2)


def rmse(y_true, y_pred):
    return np.sqrt(mse(y_true, y_pred))


def mape(y_true, y_pred, eps=1e-8):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    denom = np.maximum(np.abs(y_true), eps)
    return np.mean(np.abs(y_true - y_pred) / denom) * 100.0


def smape(y_true, y_pred, eps=1e-8):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    denom = np.maximum(np.abs(y_true) + np.abs(y_pred), eps)
    return np.mean(2.0 * np.abs(y_true - y_pred) / denom) * 100.0


EXOG_COMBOS = [
    [],
    ["event_flag"],
    ["holiday_count"],
    ["rainfall_lag1"],
    ["event_flag", "holiday_count"],
    ["event_flag", "rainfall_lag1"],
    ["holiday_count", "rainfall_lag1"],
    ["event_flag", "holiday_count", "rainfall_lag1"],
]


def fit_sarimax_one_series(
    g: pd.DataFrame,
    prof_row: pd.Series,
    seasonal_period: int = 12,
    max_p: int = 2,
    max_q: int = 2,
):
    cab = g["cabang"].iloc[0]
    sku = g["sku"].iloc[0]

    g = g.sort_values("periode").reset_index(drop=True)

    y_level = g["qty"].astype(float)
    y_log = np.log1p(y_level)

    train_mask = g["is_train"] == 1
    test_mask  = g["is_test"] == 1

    y_train = y_log[train_mask]
    y_test  = y_log[test_mask]

    if len(y_train) < 24:
        print(f"{cab}-{sku}: train terlalu pendek ({len(y_train)}), skip.")
        return None

    d_sug = int(prof_row.get("sarimax_d_sug", 1))
    D_sug = int(prof_row.get("sarimax_D_sug", 0))
    seasonal_flag = int(prof_row.get("seasonal_flag", 0))

    if seasonal_flag == 0:
        P_opts = [0]
        Q_opts = [0]
        D_sug  = 0
    else:
        P_opts = [0, 1]
        Q_opts = [0, 1]

    best_result = None
    best_res = None
    best_exog_key = "none"
    best_X_test = None

    def is_better_train(new, old):
        if old is None:
            return True
        # prioritas: train_rmse lebih kecil, kalau mirip pakai AIC
        if new["train_rmse"] < old["train_rmse"] - 1e-6:
            return True
        if abs(new["train_rmse"] - old["train_rmse"]) <= 1e-6:
            return new["aic"] < old["aic"]
        return False

    for exog_cols in EXOG_COMBOS:
        if exog_cols:
            X_all = g[exog_cols].copy()
            X_train = X_all.loc[train_mask].copy()
            X_test  = X_all.loc[test_mask].copy()

            nunique_train = X_train.nunique(dropna=True)
            keep_cols = [c for c in exog_cols if nunique_train.get(c, 0) > 1]

            if not keep_cols:
                X_all = None
                X_train = None
                X_test = None
                exog_key = "none"
            else:
                X_all = X_all[keep_cols]
                X_train = X_train[keep_cols]
                X_test  = X_test[keep_cols]
                exog_key = "+".join(keep_cols)
        else:
            X_all = None
            X_train = None
            X_test = None
            exog_key = "none"

        if X_train is not None and X_train.isna().any(axis=1).sum() > 0:
            keep = ~X_train.isna().any(axis=1)
            X_train = X_train.loc[keep]
            y_train_eff = y_train.loc[keep]
        else:
            y_train_eff = y_train

        if len(y_train_eff) < 20:
            continue

        for p in range(0, max_p + 1):
            for q in range(0, max_q + 1):
                order = (p, d_sug, q)

                for P in P_opts:
                    for Q in Q_opts:
                        seasonal_order = (P, D_sug, Q, seasonal_period)

                        try:
                            model = SARIMAX(
                                y_train_eff,
                                exog=X_train,
                                order=order,
                                seasonal_order=seasonal_order,
                                enforce_stationarity=False,
                                enforce_invertibility=False,
                            )
                            res = model.fit(disp=False)
                        except Exception:
                            continue

                        aic = res.aic
                        bic = res.bic

                        pred_train_log = res.get_prediction().predicted_mean
                        pred_train = np.expm1(pred_train_log)
                        y_train_level = np.expm1(y_train_eff)

                        train_mae_v   = mae(y_train_level, pred_train)
                        train_mse_v   = mse(y_train_level, pred_train)
                        train_rmse_v  = rmse(y_train_level, pred_train)
                        train_mape_v  = mape(y_train_level, pred_train)
                        train_smape_v = smape(y_train_level, pred_train)

                        cand = {
                            "cabang": cab,
                            "sku": sku,
                            "exog_cols": exog_key,
                            "order": order,
                            "seasonal_order": seasonal_order,
                            "aic": aic,
                            "bic": bic,
                            "train_mae": train_mae_v,
                            "train_mse": train_mse_v,
                            "train_rmse": train_rmse_v,
                            "train_mape": train_mape_v,
                            "train_smape": train_smape_v,
                        }

                        if is_better_train(cand, best_result):
                            best_result = cand
                            best_res = res
                            best_exog_key = exog_key
                            best_X_test = X_test

    if best_result is None or best_res is None:
        print(f"{cab}-{sku}: tidak ada model yang konvergen.")
        return None

    result = best_result.copy()

    test_mae_v = test_mse_v = test_rmse_v = np.nan
    test_mape_v = test_smape_v = np.nan

    if len(y_test) > 0:
        try:
            if best_exog_key == "none" or best_X_test is None or len(best_X_test) == 0:
                fc = best_res.get_forecast(steps=len(y_test))
            else:
                fc = best_res.get_forecast(steps=len(y_test), exog=best_X_test)

            pred_test_log = fc.predicted_mean
            pred_test = np.expm1(pred_test_log)
            y_test_level = np.expm1(y_test.values)

            test_mae_v   = mae(y_test_level, pred_test)
            test_mse_v   = mse(y_test_level, pred_test)
            test_rmse_v  = rmse(y_test_level, pred_test)
            test_mape_v  = mape(y_test_level, pred_test)
            test_smape_v = smape(y_test_level, pred_test)
        except Exception:
            pass

    result["test_mae"]   = test_mae_v
    result["test_mse"]   = test_mse_v
    result["test_rmse"]  = test_rmse_v
    result["test_mape"]  = test_mape_v
    result["test_smape"] = test_smape_v

    return result


def refit_and_plot_pred(g: pd.DataFrame,
                        order,
                        seasonal_order,
                        exog_key: str,
                        save_path: Path):
    cab = g["cabang"].iloc[0]
    sku = g["sku"].iloc[0]

    g = g.sort_values("periode").reset_index(drop=True)

    y_level = g["qty"].astype(float)
    y_log = np.log1p(y_level)

    train_mask = g["is_train"] == 1
    test_mask  = g["is_test"] == 1

    y_train = y_log[train_mask]
    y_test  = y_log[test_mask]

    if len(y_train) == 0:
        return

    if exog_key == "none" or exog_key == "" or exog_key is None:
        X_train = None
        X_test = None
    else:
        cols = exog_key.split("+")
        X_all = g[cols].copy()
        X_train = X_all.loc[train_mask].copy()
        X_test  = X_all.loc[test_mask].copy()

    model = SARIMAX(
        y_train,
        exog=X_train,
        order=tuple(order),
        seasonal_order=tuple(seasonal_order),
        enforce_stationarity=False,
        enforce_invertibility=False,
    )
    res = model.fit(disp=False)

    pred_train_log = res.get_prediction().predicted_mean
    pred_train = np.expm1(pred_train_log)
    y_train_level = np.expm1(y_train)

    if len(y_test) > 0:
        if X_test is not None and len(X_test) > 0:
            fc = res.get_forecast(steps=len(y_test), exog=X_test)
        else:
            fc = res.get_forecast(steps=len(y_test))
        pred_test_log = fc.predicted_mean
        pred_test = np.expm1(pred_test_log)
        y_test_level = np.expm1(y_test)
    else:
        pred_test = None
        y_test_level = None

    periode_train = g.loc[train_mask, "periode"]
    periode_test  = g.loc[test_mask, "periode"]

    fig, ax = plt.subplots(figsize=(10, 4))

    ax.plot(periode_train, y_train_level, marker="o", label="actual_train")
    ax.plot(periode_train, pred_train, marker="x", label="pred_train")

    if pred_test is not None and len(periode_test) > 0:
        ax.plot(periode_test, y_test_level, marker="o", label="actual_test")
        ax.plot(periode_test, pred_test, marker="x", label="pred_test")
        if len(periode_train) > 0:
            last_train = periode_train.max()
            ax.axvline(last_train, color="gray", linestyle="--", linewidth=1)

    all_vals = list(y_train_level.values)
    if y_test_level is not None:
        all_vals.extend(list(y_test_level.values))
    if len(all_vals) > 0:
        upper = np.percentile(all_vals, 99)
        if upper > 0:
            ax.set_ylim(0, upper * 1.1)

    ax.set_title(f"{cab} - {sku} (qty, train vs test)")
    ax.set_ylabel("qty")
    ax.legend()
    ax.grid(alpha=0.3)

    fig.tight_layout()
    fig.savefig(save_path, dpi=120)
    plt.close(fig)


results = []

grouped = panel.groupby(["cabang", "sku"], sort=False)

for (cab, sku), g in grouped:
    prof_row = profiles[(profiles["cabang"] == cab) & (profiles["sku"] == sku)]
    if prof_row.empty:
        print(f"Tidak ada profile untuk {cab}-{sku}, skip.")
        continue
    prof_row = prof_row.iloc[0]

    print(f"\nFitting {cab}-{sku}")
    res = fit_sarimax_one_series(g, prof_row)
    if res is not None:
        results.append(res)

metrics_df = pd.DataFrame(results)

metrics_df["gap_RMSE"] = metrics_df["test_rmse"] - metrics_df["train_rmse"]
metrics_df["ratio_RMSE"] = metrics_df["test_rmse"] / metrics_df["train_rmse"]

metrics_path = OUT_DIR / "sarimax_15_gridsearch_metrics.csv"
metrics_df.to_csv(metrics_path, index=False)

print("\nSaved metrics to:", metrics_path)
print("\nPreview metrics:")
print(metrics_df.head())

has_test = metrics_df[~metrics_df["test_rmse"].isna()].copy()
overfit_path = OUT_DIR / "sarimax_train_vs_test_rmse_scatter.png"

if not has_test.empty:
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.scatter(has_test["train_rmse"], has_test["test_rmse"])
    lim_min = 0
    lim_max = max(has_test["train_rmse"].max(), has_test["test_rmse"].max()) * 1.1
    ax.plot([lim_min, lim_max], [lim_min, lim_max], linestyle="--")
    ax.set_xlabel("RMSE train")
    ax.set_ylabel("RMSE test")
    ax.set_title("Train vs Test RMSE (cek overfit)")
    ax.grid(alpha=0.3)
    fig.tight_layout()
    fig.savefig(overfit_path, dpi=120)
    plt.close(fig)
    print("Saved overfit scatter:", overfit_path)

for _, row in has_test.iterrows():
    cab = row["cabang"]
    sku = row["sku"]
    order = row["order"]
    seasonal_order = row["seasonal_order"]
    exog_key = row["exog_cols"]

    g = panel[(panel["cabang"] == cab) & (panel["sku"] == sku)].copy()
    save_path = PLOT_DIR / f"pred_vs_actual_{cab}_{sku}.png"
    print("Plot pred vs actual:", cab, sku)
    refit_and_plot_pred(g, order, seasonal_order, exog_key, save_path)

print("\nSelesai SARIMAX 15 grid search + metrics + plots.")


Rows panel 15 : 4965
Rows profiles : 120
NaN rainfall_lag1: 15
Series with test rows: 9

Fitting 02A-BNOP400CHAR

Fitting 02A-BNOP400CPOX

Fitting 02A-BUVW001K194

Fitting 02A-BUVW001KSB

Fitting 02A-BUVW001KSBM

Fitting 02A-BUVW001KSW

Fitting 02A-BUVW100C192

Fitting 02A-BUVW100CSB

Fitting 02A-BUVW100CSW

Fitting 02A-CKLM001KS607

Fitting 02A-CKLM001KSPOL

Fitting 02A-DOPQ001K001

Fitting 02A-DOPQ001K009

Fitting 02A-FSTU001KSB

Fitting 02A-FSTU001KSW

Fitting 05A-BBCD005KSW

Fitting 05A-BUVW001K194

Fitting 05A-BUVW001KSB

Fitting 05A-BUVW001KSW

Fitting 05A-BUVW100C192

Fitting 05A-BUVW100CSB

Fitting 05A-BUVW100CSW

Fitting 05A-CKLM001KS600

Fitting 05A-CKLM001KS607

Fitting 05A-CKLM001KS610

Fitting 05A-DKLM350C

Fitting 05A-DOPQ001K009

Fitting 05A-DOPQ004K009

Fitting 05A-FIJK250C

Fitting 05A-FSTU001KSW

Fitting 13A-BBCD005KSW

Fitting 13A-BUVW001KSB

Fitting 13A-BUVW001KSW

Fitting 13A-BUVW100CSB

Fitting 13A-BUVW100CSW

Fitting 13A-DKLM350C

Fitting 13A-DOPQ001K002

Fitting

cek biang kerok datanya 

In [15]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# =========================================
# PATH
# =========================================
PROJECT_ROOT   = Path(r"D:\Documents\Skripsi\demand-forecasting")
DATASET15_DIR  = PROJECT_ROOT / "data" / "dataset_15"
OUT_DIR        = PROJECT_ROOT / "outputs" / "sarimax_15_gridsearch_final"

PANEL_PATH   = DATASET15_DIR / "panel_exog_selected15.csv"
METRICS_PATH = OUT_DIR / "sarimax_15_gridsearch_metrics.csv"

DIAG_DIR     = OUT_DIR / "diagnostics_drama"
DIAG_DIR.mkdir(parents=True, exist_ok=True)

# =========================================
# LOAD DATA
# =========================================
panel = pd.read_csv(PANEL_PATH, parse_dates=["periode"])
metrics = pd.read_csv(METRICS_PATH)

print("Rows panel:", len(panel))
print("Rows metrics:", len(metrics))


# =========================================
# 1. HEALTH STAT PER (CABANG, SKU) DI TRAIN
# =========================================
def build_health(df: pd.DataFrame) -> pd.DataFrame:
    g = df[df["is_train"] == 1].copy()
    g["qty"] = g["qty"].astype(float)

    agg = (
        g.groupby(["cabang", "sku"])["qty"]
         .agg(
             n_train="count",
             qty_min="min",
             qty_max="max",
             qty_median="median",
             qty_mean="mean",
             qty_std="std",
         )
         .reset_index()
    )

    # hindari bagi nol
    eps = 1e-8
    agg["max_over_median"] = agg["qty_max"] / (agg["qty_median"].abs() + eps)
    agg["max_over_mean"]   = agg["qty_max"] / (agg["qty_mean"].abs() + eps)
    agg["cv"]              = agg["qty_std"] / (agg["qty_mean"].abs() + eps)

    return agg


health_df = build_health(panel)
health_path = DIAG_DIR / "sarimax_15_health_train.csv"
health_df.to_csv(health_path, index=False)
print("Saved health stats:", health_path)


# =========================================
# 2. TRAIN vs TEST STATS PER SERIES
# =========================================
def build_train_test_stats(df: pd.DataFrame) -> pd.DataFrame:
    rows = []
    for (cab, sku), g in df.groupby(["cabang", "sku"]):
        g = g.copy()
        g["qty"] = g["qty"].astype(float)

        tr = g[g["is_train"] == 1]["qty"]
        te = g[g["is_test"] == 1]["qty"]

        if len(tr) == 0:
            continue

        row = {
            "cabang": cab,
            "sku": sku,
            "train_mean": tr.mean(),
            "train_std": tr.std(),
            "train_max": tr.max(),
            "train_min": tr.min(),
            "test_mean": np.nan,
            "test_std": np.nan,
            "test_max": np.nan,
            "test_min": np.nan,
            "n_test": len(te),
        }

        if len(te) > 0:
            row.update(
                test_mean=te.mean(),
                test_std=te.std(),
                test_max=te.max(),
                test_min=te.min(),
            )

        rows.append(row)

    return pd.DataFrame(rows)


tt_stats_df = build_train_test_stats(panel)
tt_stats_path = DIAG_DIR / "sarimax_15_train_vs_test_stats.csv"
tt_stats_df.to_csv(tt_stats_path, index=False)
print("Saved train vs test stats:", tt_stats_path)


# =========================================
# 3. MERGE: METRICS + HEALTH + TRAIN/TEST STATS
# =========================================
diag_df = (
    metrics
    .merge(health_df, on=["cabang", "sku"], how="left")
    .merge(tt_stats_df, on=["cabang", "sku"], how="left")
)

# pastikan gap_RMSE & ratio_RMSE ada (kalau skrip lama belum hitung)
if "gap_RMSE" not in diag_df.columns:
    diag_df["gap_RMSE"] = diag_df["test_rmse"] - diag_df["train_rmse"]

if "ratio_RMSE" not in diag_df.columns:
    diag_df["ratio_RMSE"] = diag_df["test_rmse"] / diag_df["train_rmse"]

diag_full_path = DIAG_DIR / "sarimax_15_diag_full.csv"
diag_df.to_csv(diag_full_path, index=False)
print("Saved full diagnostics:", diag_full_path)


# =========================================
# 4. FILTER SERIES "DRAMA"
#    - max_over_median >= 3
#    - atau cv >= 0.8
#    - atau train_rmse di top 10%
# =========================================
train_rmse_q90 = diag_df["train_rmse"].quantile(0.90)

drama_mask = (
    (diag_df["max_over_median"] >= 3) |
    (diag_df["cv"] >= 0.8) |
    (diag_df["train_rmse"] >= train_rmse_q90)
)

drama_df = diag_df.loc[drama_mask].copy()
drama_df = drama_df.sort_values("train_rmse", ascending=False)

drama_path = DIAG_DIR / "sarimax_15_drama_series.csv"
drama_df.to_csv(drama_path, index=False)
print("Saved drama series:", drama_path)
print("Jumlah series drama:", len(drama_df))


# =========================================
# 5. PLOT QTY UNTUK SERIES DRAMA (TOP N)
# =========================================
def plot_series_qty(df_panel: pd.DataFrame,
                    cabang: str,
                    sku: str,
                    out_dir: Path):
    g = df_panel[(df_panel["cabang"] == cabang) & (df_panel["sku"] == sku)].copy()
    if g.empty:
        return

    g = g.sort_values("periode")
    g["qty"] = g["qty"].astype(float)

    train_mask = g["is_train"] == 1
    test_mask  = g["is_test"] == 1

    periode = g["periode"]
    qty = g["qty"]

    fig, ax = plt.subplots(figsize=(9, 3))
    ax.plot(periode, qty, marker="o")

    if train_mask.any():
        last_train_date = g.loc[train_mask, "periode"].max()
        ax.axvline(last_train_date, color="grey", linestyle="--", linewidth=1)

    ax.set_title(f"{cabang} - {sku} (qty bulanan)")
    ax.set_ylabel("qty")
    ax.grid(alpha=0.3)
    fig.tight_layout()

    fname = f"qty_{cabang}_{sku}.png"
    save_path = out_dir / fname
    fig.savefig(save_path, dpi=120)
    plt.close(fig)
    print("Saved plot:", save_path)


PLOT_DIR = DIAG_DIR / "plots_qty_drama"
PLOT_DIR.mkdir(parents=True, exist_ok=True)

# plot beberapa seri paling drama (misal 15 teratas)
TOP_N = 15
for _, row in drama_df.head(TOP_N).iterrows():
    plot_series_qty(panel, row["cabang"], row["sku"], PLOT_DIR)

print("Selesai bikin diagnostics dan plot untuk series drama.")


Rows panel: 4965
Rows metrics: 120
Saved health stats: D:\Documents\Skripsi\demand-forecasting\outputs\sarimax_15_gridsearch_final\diagnostics_drama\sarimax_15_health_train.csv
Saved train vs test stats: D:\Documents\Skripsi\demand-forecasting\outputs\sarimax_15_gridsearch_final\diagnostics_drama\sarimax_15_train_vs_test_stats.csv
Saved full diagnostics: D:\Documents\Skripsi\demand-forecasting\outputs\sarimax_15_gridsearch_final\diagnostics_drama\sarimax_15_diag_full.csv
Saved drama series: D:\Documents\Skripsi\demand-forecasting\outputs\sarimax_15_gridsearch_final\diagnostics_drama\sarimax_15_drama_series.csv
Jumlah series drama: 44
Saved plot: D:\Documents\Skripsi\demand-forecasting\outputs\sarimax_15_gridsearch_final\diagnostics_drama\plots_qty_drama\qty_02A_BUVW001KSW.png
Saved plot: D:\Documents\Skripsi\demand-forecasting\outputs\sarimax_15_gridsearch_final\diagnostics_drama\plots_qty_drama\qty_05A_BBCD005KSW.png
Saved plot: D:\Documents\Skripsi\demand-forecasting\outputs\sarimax_

In [18]:
from pathlib import Path
import pandas as pd
import numpy as np

# ======================================================
# PATH & LOAD DATA
# ======================================================
PROJECT_ROOT   = Path(r"D:\Documents\Skripsi\demand-forecasting")
DATASET15_DIR  = PROJECT_ROOT / "data" / "dataset_15"
PANEL_PATH     = DATASET15_DIR / "panel_exog_selected15.csv"

panel = pd.read_csv(PANEL_PATH, parse_dates=["periode"])

print("Rows panel_exog_selected15:", len(panel))
print(panel.head())

# ======================================================
# FILTER TRAIN ONLY (sesuai skema kamu)
# ======================================================
train = panel[panel["is_train"] == 1].copy()

# Kalau kamu mau pastikan hanya 15 SKU terpilih:
# train = train[train["selected15"] == 1].copy()

group_cols = ["cabang", "sku"]

# ======================================================
# 1) STATISTIK PER (CABANG, SKU)
# ======================================================
agg_stats = (
    train
    .groupby(group_cols)["qty"]
    .agg(
        n_train   = "size",
        qty_min   = "min",
        qty_max   = "max",
        qty_median= "median",
        qty_mean  = "mean",
        qty_std   = "std"
    )
    .reset_index()
)

# Turunan: max / median, max / mean, CV
agg_stats["max_over_median"] = agg_stats["qty_max"] / agg_stats["qty_median"].replace(0, np.nan)
agg_stats["max_over_mean"]   = agg_stats["qty_max"] / agg_stats["qty_mean"].replace(0, np.nan)
agg_stats["cv"]              = agg_stats["qty_std"] / agg_stats["qty_mean"].replace(0, np.nan)

# ======================================================
# 2) HITUNG JUMLAH BULAN DENGAN QTY > 3x MEDIAN
# ======================================================

# temp: tempel median ke level baris bulanan
train = train.merge(
    agg_stats[group_cols + ["qty_median", "n_train"]],
    on=group_cols,
    how="left"
)

# Flag spike 3x median
train["is_spike_3x_median"] = train["qty"] > (3 * train["qty_median"])

spike_stats = (
    train
    .groupby(group_cols)["is_spike_3x_median"]
    .agg(n_spike_3x_median="sum")
    .reset_index()
)

# Merge balik ke summary
merged = agg_stats.merge(spike_stats, on=group_cols, how="left")

merged["n_spike_3x_median"] = merged["n_spike_3x_median"].fillna(0).astype(int)
merged["prop_spike_3x_median"] = merged["n_spike_3x_median"] / merged["n_train"]

# ======================================================
# 3) LIST SKU PALING DRAMA
# ======================================================

top_spiky = (
    merged
    .sort_values(
        ["n_spike_3x_median", "prop_spike_3x_median", "max_over_median"],
        ascending=[False, False, False]
    )
)

print("\nTop 15 SKU paling drama (banyak bulan qty > 3x median):")
print(
    top_spiky[
        [
            "cabang", "sku",
            "n_train",
            "qty_median", "qty_mean", "qty_max",
            "n_spike_3x_median", "prop_spike_3x_median",
            "max_over_median", "max_over_mean", "cv"
        ]
    ]
    .head(15)
    .to_string(index=False)
)

# Opsional: hanya SKU yang punya >= 3 bulan spike
spiky_only = top_spiky[top_spiky["n_spike_3x_median"] >= 3]

print("\nSKU dengan >= 3 bulan qty > 3x median:")
print(
    spiky_only[
        [
            "cabang", "sku",
            "n_train",
            "qty_median", "qty_mean", "qty_max",
            "n_spike_3x_median", "prop_spike_3x_median",
            "max_over_median", "max_over_mean", "cv"
        ]
    ]
    .to_string(index=False)
)


Rows panel_exog_selected15: 4965
     periode area cabang          sku     qty  imputed  is_active train_start  \
0 2021-01-01   1A    02A  BNOP400CHAR   412.0        0          1  2021-01-01   
1 2021-02-01   1A    02A  BNOP400CHAR   441.0        0          1  2021-01-01   
2 2021-03-01   1A    02A  BNOP400CHAR  1760.0        0          1  2021-01-01   
3 2021-04-01   1A    02A  BNOP400CHAR   502.0        0          1  2021-01-01   
4 2021-05-01   1A    02A  BNOP400CHAR   228.0        0          1  2021-01-01   

      nz_last  is_train  ...  qty_rollmean_12  qty_rollstd_12  spike_flag  \
0  2024-05-01         1  ...              NaN             NaN           0   
1  2024-05-01         1  ...           412.00             NaN           0   
2  2024-05-01         1  ...           426.50       20.506097           0   
3  2024-05-01         1  ...           871.00      770.033116           0   
4  2024-05-01         1  ...           778.75      655.241113           0   

   sample_weight 

coba lg tambah spike sebagai exog

In [20]:
# =========================================================
# SARIMAX BASELINE + ROBUST + SPIKE_FLAG EXOG
# - Data: panel_exog_selected15.csv
# - Kombinasi exog:
#     (1) none
#     (2) event_flag
#     (3) event_flag + holiday_count
#     (4) event_flag + holiday_count + rainfall_lag1
#     (5) event_flag + holiday_count + rainfall_lag1 + spike_flag
# - Target mode:
#     - "level"          : qty
#     - "log1p"          : log1p(qty)
#     - "robust_log1p"   : log1p(qty_cap)  [qty_cap = min(qty, 3×median_train) di train]
# - Evaluasi SELALU di level qty asli
# =========================================================

import warnings
import itertools
from pathlib import Path

import numpy as np
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tools.sm_exceptions import ConvergenceWarning, ValueWarning

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=ValueWarning)

# ---------------- PATH ----------------
PROJECT_ROOT   = Path(r"D:\Documents\Skripsi\demand-forecasting")
DATASET15_DIR  = PROJECT_ROOT / "data" / "dataset_15"
OUT_DIR        = PROJECT_ROOT / "outputs" / "sarimax_15_robust_spike"
OUT_DIR.mkdir(parents=True, exist_ok=True)

PANEL_PATH   = DATASET15_DIR / "panel_exog_selected15.csv"
PROFILE_PATH = DATASET15_DIR / "model_profiles_selected15.csv"  # kalau mau pakai d_suggest

# ---------------- LOAD DATA ----------------
panel = pd.read_csv(PANEL_PATH, parse_dates=["periode"])
profiles = pd.read_csv(PROFILE_PATH)

print("Rows panel_exog_selected15:", len(panel))

# Kita pakai hanya SKU yang selected15 == 1 dan eligible_model == 1
panel = panel[(panel["selected15"] == 1) & (panel["eligible_model"] == 1)].copy()

# ---------------- ROBUST: HITUNG MEDIAN TRAIN + CAP ----------------
train_stats = (
    panel[panel["is_train"] == 1]
    .groupby(["cabang", "sku"])["qty"]
    .median()
    .rename("qty_median_train")
    .reset_index()
)

panel = panel.merge(train_stats, on=["cabang", "sku"], how="left")

factor = 3.0
panel["qty_cap"] = np.where(
    panel["is_train"] == 1,
    np.minimum(panel["qty"], factor * panel["qty_median_train"]),
    panel["qty"]  # test tetap pakai qty asli
)

# ---------------- PARAM: GRID ORDER + EXOG SET ----------------

# Kalau kamu punya grid sendiri, ganti bagian ini
ORDERS = [
    (0, 1, 1),
    (0, 1, 2),
    (1, 1, 1),
    (1, 1, 2),
]

SEASONAL_ORDERS = [
    (0, 0, 0, 12),
    (0, 1, 1, 12),
]

EXOG_SETS = {
    "none": [],
    "event": ["event_flag"],
    "event_holiday": ["event_flag", "holiday_count"],
    "event_holiday_rain": ["event_flag", "holiday_count", "rainfall_lag1"],
    "event_holiday_rain_spike": ["event_flag", "holiday_count", "rainfall_lag1", "spike_flag"],
}

TARGET_MODES = ["level", "log1p", "robust_log1p"]  # robust tambahannya

# ---------------- METRIC HELPER ----------------

def calc_metrics(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)

    mae = np.mean(np.abs(y_true - y_pred))
    mse = np.mean((y_true - y_pred) ** 2)
    rmse = np.sqrt(mse)

    # MAPE: abaikan bulan dengan qty 0
    mask = y_true > 0
    if mask.sum() > 0:
        mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    else:
        mape = np.nan

    # sMAPE
    denom = np.abs(y_true) + np.abs(y_pred)
    smape = np.mean(
        np.where(
            denom == 0,
            0,
            2 * np.abs(y_true - y_pred) / denom
        )
    ) * 100

    return mae, mse, rmse, mape, smape

# ---------------- LOOP PER SERI ----------------

results = []

# Ambil daftar seri dari profiles biar konsisten
keys = profiles[["cabang", "sku"]].drop_duplicates()

for _, row in keys.iterrows():
    cabang = row["cabang"]
    sku    = row["sku"]

    df_cs = panel[(panel["cabang"] == cabang) & (panel["sku"] == sku)].sort_values("periode")
    if df_cs.empty:
        continue

    train_df = df_cs[df_cs["is_train"] == 1]
    test_df  = df_cs[df_cs["is_test"] == 1]

    if len(train_df) < 24 or len(test_df) == 0:
        # malas berantem dengan seri yang pendek
        continue

    y_train_level = train_df["qty"].astype(float)
    y_train_log   = np.log1p(y_train_level.clip(lower=0))
    y_train_cap   = train_df["qty_cap"].astype(float)
    y_train_robust_log = np.log1p(y_train_cap.clip(lower=0))

    y_test = test_df["qty"].astype(float)

    # optional: ambil d_suggest kalau mau nyocokkan order dengan profil
    prof_row = profiles[(profiles["cabang"] == cabang) & (profiles["sku"] == sku)]
    if not prof_row.empty and "suggest_d" in prof_row.columns:
        d_suggest = int(prof_row["suggest_d"].iloc[0])
    else:
        d_suggest = 1

    # filter ORDERS yang d-nya sesuai d_suggest, kalau ingin
    orders_use = [o for o in ORDERS if o[1] == d_suggest] or ORDERS

    for order, seasonal_order, (exog_name, exog_cols), target_mode in itertools.product(
        orders_use,
        SEASONAL_ORDERS,
        EXOG_SETS.items(),
        TARGET_MODES
    ):
        # siapkan exog
        if exog_cols:
            exog_train = train_df[exog_cols].astype(float)
            exog_test  = test_df[exog_cols].astype(float)
        else:
            exog_train = None
            exog_test  = None

        # pilih target train sesuai mode
        if target_mode == "level":
            y_train = y_train_level
            use_log = False
            use_robust = False
        elif target_mode == "log1p":
            y_train = y_train_log
            use_log = True
            use_robust = False
        elif target_mode == "robust_log1p":
            y_train = y_train_robust_log
            use_log = True
            use_robust = True
        else:
            continue

        try:
            model = SARIMAX(
                endog=y_train,
                exog=exog_train,
                order=order,
                seasonal_order=seasonal_order,
                enforce_stationarity=False,
                enforce_invertibility=False,
            )
            res = model.fit(disp=False)

            # forecasting untuk test
            n_test = len(test_df)
            fc = res.get_forecast(steps=n_test, exog=exog_test)
            y_pred_raw = fc.predicted_mean

            if use_log:
                y_pred = np.expm1(y_pred_raw)
                y_pred = np.clip(y_pred, a_min=0, a_max=None)
            else:
                y_pred = y_pred_raw

            mae, mse, rmse, mape, smape = calc_metrics(y_test, y_pred)

            results.append({
                "cabang": cabang,
                "sku": sku,
                "order": order,
                "seasonal_order": seasonal_order,
                "exog_name": exog_name,
                "target_mode": target_mode,
                "aic": res.aic,
                "bic": res.bic,
                "train_n": len(train_df),
                "test_n": len(test_df),
                "train_end": train_df["periode"].max(),
                "test_start": test_df["periode"].min(),
                "test_end": test_df["periode"].max(),
                "test_mae": mae,
                "test_mse": mse,
                "test_rmse": rmse,
                "test_mape": mape,
                "test_smape": smape,
            })

        except Exception as e:
            # kalau seri ini kombinasi tertentu meledak, abaikan
            print(f"Skip {cabang} {sku} | order={order} seas={seasonal_order} exog={exog_name} target={target_mode} | err={e}")
            continue

# ---------------- SIMPAN HASIL ----------------
results_df = pd.DataFrame(results)
out_path = OUT_DIR / "sarimax_15_robust_spike_results.csv"
results_df.to_csv(out_path, index=False)
print("Selesai. Rows hasil:", len(results_df))
print("Saved to:", out_path)


Rows panel_exog_selected15: 4965
Skip 16C DOPQ001K009 | order=(0, 1, 1) seas=(0, 0, 0, 12) exog=event_holiday_rain target=level | err=exog contains inf or nans
Skip 16C DOPQ001K009 | order=(0, 1, 1) seas=(0, 0, 0, 12) exog=event_holiday_rain target=log1p | err=exog contains inf or nans
Skip 16C DOPQ001K009 | order=(0, 1, 1) seas=(0, 0, 0, 12) exog=event_holiday_rain target=robust_log1p | err=exog contains inf or nans
Skip 16C DOPQ001K009 | order=(0, 1, 1) seas=(0, 0, 0, 12) exog=event_holiday_rain_spike target=level | err=exog contains inf or nans
Skip 16C DOPQ001K009 | order=(0, 1, 1) seas=(0, 0, 0, 12) exog=event_holiday_rain_spike target=log1p | err=exog contains inf or nans
Skip 16C DOPQ001K009 | order=(0, 1, 1) seas=(0, 0, 0, 12) exog=event_holiday_rain_spike target=robust_log1p | err=exog contains inf or nans
Skip 16C DOPQ001K009 | order=(0, 1, 1) seas=(0, 1, 1, 12) exog=event_holiday_rain target=level | err=exog contains inf or nans
Skip 16C DOPQ001K009 | order=(0, 1, 1) seas=(0